# 047: ML Pipelines & Automation## 🎯 Learning ObjectivesBy the end of this notebook, you will:1. **Understand ML pipelines** - Why pipelines matter, composition, reproducibility2. **Master sklearn Pipeline** - Sequential transformers, ColumnTransformer, FeatureUnion3. **Build custom transformers** - Extend BaseEstimator, fit/transform pattern, stateful transformers4. **Implement end-to-end pipelines** - Data ingestion → preprocessing → model → evaluation5. **Handle complex workflows** - Conditional transformations, parallel processing, caching6. **Apply to post-silicon validation** - STDF data pipelines, multi-stage test flows7. **Production patterns** - Serialization, versioning, monitoring, CI/CD integration8. **Automate repetitive tasks** - Grid search over pipelines, automated feature engineering---## 📊 Why ML Pipelines Matter```mermaidgraph LR    A[Raw Data] --> B[Preprocessing]    B --> C[Feature Engineering]    C --> D[Model Training]    D --> E[Prediction]        style A fill:#ffe6e6    style B fill:#e6f3ff    style C fill:#e6f3ff    style D fill:#e6ffe6    style E fill:#90EE90        F[Without Pipeline:<br/>Scattered Code] -.-> G[❌ Data Leakage<br/>❌ Irreproducible<br/>❌ Error-Prone]        H[With Pipeline:<br/>Unified Object] -.-> I[✅ No Leakage<br/>✅ Reproducible<br/>✅ Production-Ready]```---### **The Problem: Scattered Preprocessing****Typical messy workflow:**```python# TrainX_train_scaled = scaler.fit_transform(X_train)X_train_encoded = encoder.fit_transform(X_train_scaled)model.fit(X_train_encoded, y_train)# Test (OOPS! Forgot to apply same transforms!)X_test_scaled = scaler.transform(X_test)  # Easy to forgetX_test_encoded = encoder.transform(X_test_scaled)  # Error-proney_pred = model.predict(X_test_encoded)# Production (nightmare!)# How do I remember the exact sequence?# Did I fit scaler on train or all data? (data leakage risk!)```**Issues:**- ❌ **Data leakage:** Accidentally fit scaler on test data- ❌ **Inconsistency:** Different transforms for train/test/production- ❌ **Irreproducible:** Can't recreate exact preprocessing steps- ❌ **Error-prone:** Forget a step → silent model performance degradation- ❌ **Hard to serialize:** How to save model + all preprocessing steps?---### **The Solution: Unified Pipeline****Clean pipeline workflow:**```pythonfrom sklearn.pipeline import Pipeline# Define oncepipeline = Pipeline([    ('scaler', StandardScaler()),    ('encoder', OneHotEncoder()),    ('model', RandomForestClassifier())])# Train (fit pipeline = fit all steps on train data)pipeline.fit(X_train, y_train)# Test (transform pipeline = apply all steps consistently)y_pred = pipeline.predict(X_test)# Production (single object, no mistakes!)import joblibjoblib.dump(pipeline, 'model_v1.pkl')loaded_pipeline = joblib.load('model_v1.pkl')y_prod = loaded_pipeline.predict(X_new)  # All transforms applied correctly!```**Benefits:**- ✅ **No data leakage:** Scaler fit only on train, transform on test- ✅ **Consistency:** Exact same transforms for train/test/production- ✅ **Reproducible:** Single object captures entire workflow- ✅ **Easy serialization:** Save once, deploy anywhere- ✅ **Grid search compatible:** Tune preprocessing + model simultaneously---## 🎓 Post-Silicon Validation Context### Why Pipelines Are Critical in Semiconductor Testing:1. **Multi-Stage Test Flows** ($50M-$200M ATE investment)   - Wafer test → Package → Final test → Burn-in → System test   - Each stage has different preprocessing (spatial averaging, temporal filtering, outlier removal)   - Pipeline ensures consistent transforms across stages2. **STDF Data Preprocessing** (Standard Test Data Format)   - Raw STDF: 1000+ parametric tests per device, missing data, outliers, spatial correlation   - Preprocessing: Missing data imputation → Outlier removal → Feature scaling → Spatial detrending   - Pipeline prevents: "Which outlier threshold did we use in Model v2?" (versioning nightmare)3. **Real-Time Production Deployment** (1M devices/day)   - Test equipment generates predictions in real-time (<100ms latency)   - Pipeline serialization: Train offline → deploy as single .pkl file → load on edge device   - No room for error: Wrong preprocessing → 5% yield loss → $10M-$50M annual impact4. **Regulatory Compliance** (Automotive, Medical)   - Auditors ask: "Exactly how is data preprocessed before prediction?"   - Pipeline code = documentation: Every transform is explicit, version-controlled   - ISO 26262 requirement: "ML model processing must be deterministic and traceable"---## 🔑 Core Concepts### **1. Pipeline Fundamentals****Definition:** A pipeline is a **sequence of transforms** followed by a **final estimator**.```pythonPipeline([    ('transform_1', Transformer1()),  # fit_transform on train, transform on test    ('transform_2', Transformer2()),  # fit_transform on train, transform on test    ('estimator', Model())            # fit on train, predict on test])```**Key properties:**- All intermediate steps must implement `fit()` and `transform()`- Final step must implement `fit()` and optionally `predict()` or `transform()`- Calling `pipeline.fit(X, y)` sequentially calls:  1. `transform_1.fit(X, y)`, then `X = transform_1.transform(X)`  2. `transform_2.fit(X, y)`, then `X = transform_2.transform(X)`  3. `estimator.fit(X, y)`- Calling `pipeline.predict(X)` sequentially calls:  1. `X = transform_1.transform(X)` (no fit!)  2. `X = transform_2.transform(X)` (no fit!)  3. `y = estimator.predict(X)`---### **2. ColumnTransformer: Heterogeneous Data****Problem:** Different columns need different preprocessing```python# Numeric: Scale# Categorical: One-hot encode# Text: TF-IDF vectorize# Dates: Extract features (day_of_week, month, etc.)```**Solution:** `ColumnTransformer` applies different transformers to different columns```pythonfrom sklearn.compose import ColumnTransformerpreprocessor = ColumnTransformer([    ('num', StandardScaler(), ['Vdd', 'Idd', 'freq']),       # Numeric columns    ('cat', OneHotEncoder(), ['site_id', 'product_type']),  # Categorical columns    ('passthrough', 'passthrough', ['wafer_id'])            # Keep as-is])```**Parallel execution:** Transforms run independently, then concatenate outputs---### **3. FeatureUnion: Parallel Features****Problem:** Generate multiple feature representations in parallel```python# Option A: PCA (dimensionality reduction)# Option B: SelectKBest (feature selection)# Combine both representations```**Solution:** `FeatureUnion` runs transformers in parallel, concatenates results```pythonfrom sklearn.pipeline import FeatureUnionfeature_engineering = FeatureUnion([    ('pca', PCA(n_components=50)),    ('select', SelectKBest(k=30))])# Output: 50 PCA features + 30 selected features = 80 features total```---### **4. Custom Transformers****When to write custom transformers:**- Domain-specific preprocessing (spatial detrending for wafer data)- Complex feature engineering (time-series rolling statistics)- Conditional transformations (different logic for different data ranges)**Pattern:** Inherit from `BaseEstimator` and `TransformerMixin````pythonfrom sklearn.base import BaseEstimator, TransformerMixinclass MyTransformer(BaseEstimator, TransformerMixin):    def __init__(self, param1=1.0):        self.param1 = param1        def fit(self, X, y=None):        # Learn parameters from training data        self.learned_stat_ = X.mean()  # Trailing _ = learned attribute        return self        def transform(self, X):        # Apply transformation using learned parameters        return X - self.learned_stat_```**Key rules:**- `__init__`: Only store hyperparameters (no data-dependent logic)- `fit`: Learn parameters from training data, store with trailing underscore `_`- `transform`: Apply transformation using learned parameters (no re-fitting)- `fit_transform`: Provided automatically by `TransformerMixin`---## 🛠️ When to Use Each Component| **Component** | **Use Case** | **Example** ||---------------|--------------|-------------|| **Pipeline** | Sequential transforms + model | Scaler → PCA → Random Forest || **ColumnTransformer** | Different transforms for different columns | Numeric: scale, Categorical: encode || **FeatureUnion** | Combine multiple feature representations | PCA features + Original features || **Custom Transformer** | Domain-specific preprocessing | Wafer spatial detrending, STDF outlier removal || **FunctionTransformer** | Simple stateless transforms | `np.log`, `np.sqrt` without state || **make_pipeline** | Quick pipeline without naming steps | `make_pipeline(Scaler(), Model())` |---## 🏭 Semiconductor-Specific Pipelines### **Challenge 1: Spatial Correlation on Wafers****Problem:** Adjacent dies are similar → violates IID assumption**Pipeline solution:**1. Group by wafer_id2. Compute within-wafer mean/std (spatial statistics)3. Detrend each die by subtracting wafer mean4. Feed detrended values to model**Custom transformer:** `WaferSpatialDetrending`---### **Challenge 2: Multi-Stage Test Correlation****Problem:** Wafer test + Final test data must be merged, but have different feature spaces**Pipeline solution:**1. Wafer test pipeline: 200 tests → PCA → 50 features2. Final test pipeline: 150 tests → Feature selection → 30 features3. FeatureUnion: Concatenate 50 + 30 = 80 features4. Model: XGBoost on 80 features---### **Challenge 3: Real-Time Inference Latency****Problem:** Production environment requires <50ms latency (1M devices/day)**Pipeline optimization:**1. Cache preprocessing: Fit once, save pipeline2. Batch inference: Process 1000 devices at once (vectorized)3. Feature selection: Remove low-importance features (40% speedup)4. Model simplification: Ensemble with 50 trees (vs 500 for offline)---## 📚 What We'll Build### **From Scratch (Educational):**1. **Simple Pipeline** - Manual implementation to understand internals2. **Custom Transformer** - Domain-specific preprocessing for STDF data### **Production (Practical):**3. **sklearn Pipeline** - Standard scaler + model4. **ColumnTransformer** - Heterogeneous data (numeric + categorical)5. **End-to-end pipeline** - Data ingestion → preprocessing → model → evaluation6. **Serialization & deployment** - Save/load pipeline, version control---## 🎯 Real-World Applications### **Post-Silicon Validation:**- **Automated test flow** - Wafer test → Final test → Binning (single pipeline)- **Spatial feature engineering** - Wafer map detrending + model- **Multi-site harmonization** - Site-specific preprocessing + unified model- **Real-time binning** - <50ms latency, serialized pipeline on edge device### **General AI/ML:**- **Production ML** - Text classification, fraud detection, recommendation systems- **AutoML integration** - Grid search over pipeline hyperparameters- **A/B testing** - Easy to swap pipeline versions, compare performance- **Model monitoring** - Track pipeline drift (preprocessing statistics)---**Let's begin!** 🚀

## 📐 Mathematical Foundation: Pipeline Architecture

### **Pipeline as Function Composition**

**Mathematical view:** A pipeline is a **composition of functions**

$$
\text{Pipeline}(x) = f_3(f_2(f_1(x)))
$$

Where:
- $f_1$: First transformer (e.g., StandardScaler)
- $f_2$: Second transformer (e.g., PCA)
- $f_3$: Final estimator (e.g., Logistic Regression)

---

### **Training Phase: Sequential Fitting**

**Step-by-step execution of `pipeline.fit(X_train, y_train)`:**

1. **Fit and transform first step:**
   $$
   \theta_1 = \text{fit}(f_1, X_{\text{train}}, y_{\text{train}})
   $$
   $$
   X_1 = f_1(X_{\text{train}}; \theta_1)
   $$
   
   Example: StandardScaler learns $\mu_1, \sigma_1$ from $X_{\text{train}}$, then transforms:
   $$
   X_1 = \frac{X_{\text{train}} - \mu_1}{\sigma_1}
   $$

2. **Fit and transform second step:**
   $$
   \theta_2 = \text{fit}(f_2, X_1, y_{\text{train}})
   $$
   $$
   X_2 = f_2(X_1; \theta_2)
   $$
   
   Example: PCA learns principal components $W_2$ from $X_1$, then projects:
   $$
   X_2 = X_1 \cdot W_2
   $$

3. **Fit final estimator:**
   $$
   \theta_3 = \text{fit}(f_3, X_2, y_{\text{train}})
   $$
   
   Example: Logistic Regression learns weights $\beta_3$ from $X_2$:
   $$
   \beta_3 = \arg\min_{\beta} \sum_{i=1}^{n} \log(1 + e^{-y_i (\beta^T X_2)})
   $$

**Key principle:** Each step is fit **only on the output of the previous step**, preventing data leakage.

---

### **Prediction Phase: Transform-Only**

**Step-by-step execution of `pipeline.predict(X_test)`:**

1. **Transform with first step (NO fitting):**
   $$
   X_1 = f_1(X_{\text{test}}; \theta_1)
   $$
   
   Example: Apply StandardScaler with **training statistics**:
   $$
   X_1 = \frac{X_{\text{test}} - \mu_1}{\sigma_1}
   $$
   
   ⚠️ **Critical:** Use $\mu_1, \sigma_1$ learned from training, **NOT** test statistics!

2. **Transform with second step (NO fitting):**
   $$
   X_2 = f_2(X_1; \theta_2)
   $$
   
   Example: Apply PCA with **training components**:
   $$
   X_2 = X_1 \cdot W_2
   $$

3. **Predict with final estimator:**
   $$
   \hat{y} = f_3(X_2; \theta_3)
   $$
   
   Example: Logistic Regression prediction:
   $$
   \hat{y} = \text{sign}(\beta_3^T X_2)
   $$

**Key principle:** Parameters $\theta_1, \theta_2, \theta_3$ are **frozen** during prediction.

---

### **Why This Prevents Data Leakage**

**Bad practice (manual code):**

```python
# WRONG! Scaler sees test data during fit()
scaler.fit(np.concatenate([X_train, X_test]))  # ❌ Data leakage!
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
```

**Mathematical issue:** Test statistics $\mu_{\text{test}}, \sigma_{\text{test}}$ leak into training:
$$
\mu_{\text{combined}} = \frac{n_{\text{train}} \mu_{\text{train}} + n_{\text{test}} \mu_{\text{test}}}{n_{\text{train}} + n_{\text{test}}}
$$

Result: Model has seen test data indirectly → optimistic performance estimate.

---

**Good practice (pipeline):**

```python
# CORRECT! Scaler sees only training data
pipeline.fit(X_train, y_train)  # ✅ No leakage
y_pred = pipeline.predict(X_test)
```

**Mathematical guarantee:** Parameters learned only from training:
$$
\theta_1 = \text{fit}(f_1, X_{\text{train}}, y_{\text{train}})
$$
$$
\mu_1 = \frac{1}{n_{\text{train}}} \sum_{i \in \text{train}} X_i
$$

Test data **never** influences $\mu_1$.

---

### **ColumnTransformer as Block-Diagonal Transformation**

**Mathematical structure:**

For dataset with 3 column groups (numeric, categorical, text):

$$
X = [X_{\text{num}} \mid X_{\text{cat}} \mid X_{\text{text}}]
$$

ColumnTransformer applies different functions:

$$
T(X) = [f_{\text{num}}(X_{\text{num}}) \mid f_{\text{cat}}(X_{\text{cat}}) \mid f_{\text{text}}(X_{\text{text}})]
$$

**As matrix transformation:**

$$
T = \begin{bmatrix}
T_{\text{num}} & 0 & 0 \\
0 & T_{\text{cat}} & 0 \\
0 & 0 & T_{\text{text}}
\end{bmatrix}
$$

**Block-diagonal structure:** Columns are transformed **independently**, then concatenated.

---

### **Pipeline + GridSearchCV: Joint Optimization**

**Traditional approach (suboptimal):**

```python
# Step 1: Optimize preprocessing
best_n_components = grid_search_pca(X_train)

# Step 2: Optimize model (PCA is fixed!)
pca = PCA(n_components=best_n_components)
X_train_pca = pca.fit_transform(X_train)
best_C = grid_search_logreg(X_train_pca)
```

**Problem:** PCA optimization doesn't consider model performance → suboptimal combination.

---

**Pipeline approach (optimal):**

```python
pipeline = Pipeline([('pca', PCA()), ('logreg', LogisticRegression())])

param_grid = {
    'pca__n_components': [10, 20, 50],
    'logreg__C': [0.1, 1.0, 10.0]
}

grid_search = GridSearchCV(pipeline, param_grid)
grid_search.fit(X_train, y_train)
```

**Mathematical formulation:**

$$
(\theta_1^*, \theta_3^*) = \arg\min_{n_{\text{comp}}, C} \text{CV-Error}(\text{PCA}(n_{\text{comp}}) \to \text{LogReg}(C))
$$

**Benefit:** PCA and LogisticRegression hyperparameters are optimized **jointly** for best cross-validation score.

---

### **Computational Complexity**

**Pipeline overhead:**

| **Operation** | **Without Pipeline** | **With Pipeline** | **Difference** |
|---------------|----------------------|-------------------|----------------|
| **Training** | $O(n \cdot p \cdot k)$ | $O(n \cdot p \cdot k)$ | No overhead |
| **Prediction** | $O(m \cdot p \cdot k)$ | $O(m \cdot p \cdot k)$ | No overhead |
| **Memory** | Store $k$ objects separately | Store 1 pipeline object | Cleaner |

Where:
- $n$: Training samples
- $m$: Test samples
- $p$: Features
- $k$: Number of pipeline steps

**Key insight:** Pipelines have **zero computational overhead** (just organizational benefit).

---

### **Caching: Memory-Computation Tradeoff**

**Problem:** Expensive preprocessing (e.g., TF-IDF on 1M documents)

**Solution:** Cache intermediate results

```python
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),  # Expensive: O(n * d * v)
    ('model', LogisticRegression())
], memory='/tmp/cache')
```

**Mathematical benefit:**

**Without caching (GridSearchCV with 10 folds):**
- TF-IDF computed 10 times: $10 \times O(n \cdot d \cdot v)$

**With caching:**
- TF-IDF computed once: $1 \times O(n \cdot d \cdot v)$
- Speedup: **10x** for preprocessing step

---

### **Semiconductor Example: Spatial Detrending Math**

**Problem:** Wafer maps show spatial gradients (edge dies yield < center dies)

**Mathematical model:**

1. **Raw yield:**
   $$
   Y_{i,j} = f_{\text{process}} + f_{\text{spatial}}(x_i, y_j) + \epsilon_{i,j}
   $$
   
   Where:
   - $f_{\text{process}}$: True process variation (what we want to model)
   - $f_{\text{spatial}}(x_i, y_j)$: Spatial bias (nuisance)
   - $\epsilon_{i,j}$: Random noise

2. **Spatial detrending:**
   $$
   Y_{i,j}^{\text{detrended}} = Y_{i,j} - \bar{Y}_{\text{wafer}} = f_{\text{process}} + \epsilon_{i,j}
   $$
   
   Where:
   $$
   \bar{Y}_{\text{wafer}} = \frac{1}{N_{\text{dies}}} \sum_{i,j} Y_{i,j}
   $$

3. **Pipeline implementation:**
   ```python
   class WaferSpatialDetrending(BaseEstimator, TransformerMixin):
       def fit(self, X, y=None):
           # Learn wafer-level means from training data
           self.wafer_means_ = X.groupby('wafer_id')['yield'].mean()
           return self
       
       def transform(self, X):
           # Subtract wafer-level mean from each die
           X['yield_detrended'] = X['yield'] - X['wafer_id'].map(self.wafer_means_)
           return X
   ```

**Result:** Model learns $f_{\text{process}}$ without spatial bias → better generalization.

---

### **Summary: Pipeline Mathematics**

| **Concept** | **Mathematical Essence** | **Practical Benefit** |
|-------------|-------------------------|----------------------|
| **Pipeline** | Function composition $f_3 \circ f_2 \circ f_1$ | No data leakage |
| **Sequential fitting** | $\theta_i = \text{fit}(f_i, X_{i-1}, y)$ | Each step sees only previous output |
| **Transform-only prediction** | $X_i = f_i(X_{i-1}; \theta_i)$ (no re-fitting) | Consistent preprocessing |
| **ColumnTransformer** | Block-diagonal transformation | Independent column groups |
| **GridSearchCV + Pipeline** | Joint optimization $\arg\min_{\theta_1, \theta_3}$ | Optimal hyperparameter combination |
| **Caching** | Avoid redundant computation | $k$-fold speedup |

---

**Next:** We'll implement simple pipelines from scratch to see these principles in action! 🔨

### 📝 What's Happening in This Code?

**Purpose:** Implement a **simple pipeline from scratch** to understand how sklearn pipelines work internally.

**Key Points:**
- **SimplePipeline class:** Mimics sklearn.pipeline.Pipeline with sequential fit/transform/predict
- **fit() method:** Iteratively fits each step on the output of the previous step (prevents data leakage)
- **predict() method:** Applies transform() to all steps except the last, then predict() on the final estimator
- **Naming convention:** Steps are `(name, transformer)` tuples for easy parameter access
- **Semiconductor example:** StandardScaler → PCA → LogisticRegression for yield classification

**Why This Matters:** Understanding pipeline internals helps debug issues, write custom transformers, and optimize performance. For semiconductor manufacturing, pipelines ensure consistent preprocessing across wafer test → final test → production deployment ($50M-$200M ATE investment).

### 📝 Implementation

**Purpose:** Core implementation with detailed code

**Key implementation details below.**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
# ===========================
# Simple Pipeline From Scratch
# ===========================
class SimplePipeline:
    """
    Educational implementation of sklearn Pipeline.
    
    A pipeline chains multiple transformers and a final estimator.
    During fit():
        - Each transformer is fit on the output of the previous step
    During predict():
        - Each transformer is applied (no fitting), then final estimator predicts
    
    Parameters:
    -----------
    steps : list of (name, transformer) tuples
        Sequence of transformers + final estimator
    
    Example:
    --------
    pipeline = SimplePipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=2)),
        ('classifier', LogisticRegression())
    ])
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    """
    
    def __init__(self, steps):
        self.steps = steps
        self.named_steps = {name: transformer for name, transformer in steps}
    
    def fit(self, X, y=None):
        """
        Fit all transformers sequentially, then fit final estimator.
        
        Process:
        1. Fit first transformer on X, transform to get X_1
        2. Fit second transformer on X_1, transform to get X_2
        3. ... continue for all transformers
        4. Fit final estimator on X_final
        """
        X_current = X.copy()
        
        # Fit and transform all intermediate steps
        for name, transformer in self.steps[:-1]:
            print(f"[Pipeline] Fitting {name}...")
            transformer.fit(X_current, y)
            X_current = transformer.transform(X_current)
            print(f"  Shape after {name}: {X_current.shape}")
        
        # Fit final estimator
        final_name, final_estimator = self.steps[-1]
        print(f"[Pipeline] Fitting {final_name}...")
        final_estimator.fit(X_current, y)
        print(f"  Final estimator trained!")
        
        return self
    
    def predict(self, X):
        """
        Apply all transformers (no fitting), then predict with final estimator.
        
        Process:
        1. Transform X with first transformer (using parameters learned during fit)
        2. Transform X with second transformer (using parameters learned during fit)
        3. ... continue for all transformers
        4. Predict with final estimator
        """
        X_current = X.copy()
        
        # Transform with all intermediate steps (NO FITTING!)
        for name, transformer in self.steps[:-1]:
            X_current = transformer.transform(X_current)
        
        # Predict with final estimator
        _, final_estimator = self.steps[-1]
        return final_estimator.predict(X_current)
    
    def score(self, X, y):
        """Compute accuracy score."""
        y_pred = self.predict(X)
        return accuracy_score(y, y_pred)


### 📝 Implementation Part 2

**Purpose:** Continue implementation

**Key implementation details below.**

In [ ]:
# ========================================
# Generate Semiconductor Yield Data
# ========================================
np.random.seed(42)
n_samples = 1000
n_features = 10
# Generate features: Vdd_min, Vdd_max, Idd_active, Idd_standby, freq_max, temp, etc.
feature_names = [
    'Vdd_min', 'Vdd_max', 'Idd_active', 'Idd_standby', 'freq_max',
    'temp', 'Vth', 'leakage', 'delay', 'power'
]
# True signal: yield depends on Vdd_min, Idd_active, freq_max
X_signal = np.random.randn(n_samples, 3)
X_noise = np.random.randn(n_samples, 7) * 0.5  # Less informative features
X = np.hstack([X_signal, X_noise])
# Generate binary yield labels (0 = fail, 1 = pass)
# Logistic decision boundary: yield = 1 if (0.8*Vdd_min + 0.6*Idd_active + 0.4*freq_max + noise > 0)
y_prob = 1 / (1 + np.exp(-(0.8*X[:, 0] + 0.6*X[:, 2] + 0.4*X[:, 4] - 0.5)))
y = (y_prob > 0.5).astype(int)
print("=" * 70)
print("Semiconductor Yield Classification with Pipeline")
print("=" * 70)
print(f"Dataset: {n_samples} devices, {n_features} features")
print(f"Features: {', '.join(feature_names)}")
print(f"Target: Binary yield (0 = fail, 1 = pass)")
print(f"Class distribution: {np.sum(y == 0)} fails, {np.sum(y == 1)} passes")
print()
# Split train/test
train_size = int(0.8 * n_samples)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
print(f"Train: {X_train.shape[0]} samples")
print(f"Test: {X_test.shape[0]} samples")
print()
# ========================================
# Build and Train Simple Pipeline
# ========================================
print("=" * 70)
print("Training Simple Pipeline (From Scratch)")
print("=" * 70)
# Define pipeline: StandardScaler → PCA → LogisticRegression
simple_pipeline = SimplePipeline([
    ('scaler', StandardScaler()),              # Step 1: Normalize features
    ('pca', PCA(n_components=5)),              # Step 2: Dimensionality reduction
    ('classifier', LogisticRegression(max_iter=1000))  # Step 3: Classification
])
# Train pipeline
simple_pipeline.fit(X_train, y_train)
print()
# Evaluate on test set
y_pred_train = simple_pipeline.predict(X_train)
y_pred_test = simple_pipeline.predict(X_test)
train_acc = accuracy_score(y_train, y_pred_train)
test_acc = accuracy_score(y_test, y_pred_test)
print("=" * 70)
print("Simple Pipeline Results")
print("=" * 70)
print(f"Train Accuracy: {train_acc:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")
print()


### 📝 Implementation Part 3

**Purpose:** Continue implementation

**Key implementation details below.**

In [ ]:
# ========================================
# Compare: Manual Preprocessing (Error-Prone)
# ========================================
print("=" * 70)
print("Comparison: Manual Preprocessing (Error-Prone)")
print("=" * 70)
# Manual approach: Fit scaler, transform, fit PCA, transform, fit classifier
scaler_manual = StandardScaler()
pca_manual = PCA(n_components=5)
classifier_manual = LogisticRegression(max_iter=1000)
# Train
X_train_scaled = scaler_manual.fit_transform(X_train)
X_train_pca = pca_manual.fit_transform(X_train_scaled)
classifier_manual.fit(X_train_pca, y_train)
# Test (must remember to apply same transforms!)
X_test_scaled = scaler_manual.transform(X_test)
X_test_pca = pca_manual.transform(X_test_scaled)
y_pred_manual = classifier_manual.predict(X_test_pca)
manual_test_acc = accuracy_score(y_test, y_pred_manual)
print(f"Manual Test Accuracy: {manual_test_acc:.4f}")
print(f"Pipeline Test Accuracy: {test_acc:.4f}")
print(f"Difference: {abs(manual_test_acc - test_acc):.6f} (should be ~0)")
print()
# Show potential error: Forgot to scale test data
print("⚠️ Common Error: Forgot to scale test data")
X_test_unscaled_pca = pca_manual.transform(X_test)  # OOPS! Forgot to scale
y_pred_error = classifier_manual.predict(X_test_unscaled_pca)
error_test_acc = accuracy_score(y_test, y_pred_error)
print(f"Wrong Test Accuracy (unscaled): {error_test_acc:.4f}")
print(f"Performance drop: {test_acc - error_test_acc:.4f}")
print("→ Pipeline prevents this mistake!")
print()
# ========================================
# Visualization 1: Confusion Matrix
# ========================================
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
# Pipeline confusion matrix
cm_pipeline = confusion_matrix(y_test, y_pred_test)
sns.heatmap(cm_pipeline, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['Fail', 'Pass'], yticklabels=['Fail', 'Pass'])
axes[0].set_title(f'Pipeline Confusion Matrix (Acc: {test_acc:.3f})', fontsize=12, weight='bold')
axes[0].set_xlabel('Predicted', fontsize=10)
axes[0].set_ylabel('Actual', fontsize=10)
# Manual confusion matrix
cm_manual = confusion_matrix(y_test, y_pred_manual)
sns.heatmap(cm_manual, annot=True, fmt='d', cmap='Greens', ax=axes[1],
            xticklabels=['Fail', 'Pass'], yticklabels=['Fail', 'Pass'])
axes[1].set_title(f'Manual Confusion Matrix (Acc: {manual_test_acc:.3f})', fontsize=12, weight='bold')
axes[1].set_xlabel('Predicted', fontsize=10)
axes[1].set_ylabel('Actual', fontsize=10)
plt.tight_layout()
plt.show()
print("✅ Visualization 1: Confusion matrices show identical performance (pipeline = manual when done correctly)")
print()
# ========================================
# Visualization 2: Pipeline Flow


### 📝 Implementation Part 4

**Purpose:** Continue implementation

**Key implementation details below.**

In [ ]:
# ========================================
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
# Step 1: Original data (first 2 features)
axes[0].scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap='coolwarm', alpha=0.6, edgecolors='k')
axes[0].set_title('Step 1: Original Data (Vdd_min vs Vdd_max)', fontsize=11, weight='bold')
axes[0].set_xlabel('Vdd_min (raw)', fontsize=9)
axes[0].set_ylabel('Vdd_max (raw)', fontsize=9)
axes[0].grid(alpha=0.3)
# Step 2: After scaling
scaler_viz = StandardScaler()
X_train_scaled_viz = scaler_viz.fit_transform(X_train)
axes[1].scatter(X_train_scaled_viz[:, 0], X_train_scaled_viz[:, 1], c=y_train, cmap='coolwarm', alpha=0.6, edgecolors='k')
axes[1].set_title('Step 2: After StandardScaler', fontsize=11, weight='bold')
axes[1].set_xlabel('Vdd_min (scaled)', fontsize=9)
axes[1].set_ylabel('Vdd_max (scaled)', fontsize=9)
axes[1].grid(alpha=0.3)
# Step 3: After PCA
pca_viz = PCA(n_components=5)
X_train_pca_viz = pca_viz.fit_transform(X_train_scaled_viz)
axes[2].scatter(X_train_pca_viz[:, 0], X_train_pca_viz[:, 1], c=y_train, cmap='coolwarm', alpha=0.6, edgecolors='k')
axes[2].set_title('Step 3: After PCA (2 components)', fontsize=11, weight='bold')
axes[2].set_xlabel('PC1', fontsize=9)
axes[2].set_ylabel('PC2', fontsize=9)
axes[2].grid(alpha=0.3)
plt.tight_layout()
plt.show()
print("✅ Visualization 2: Pipeline flow shows sequential transformations")
print()
# ========================================
# Key Takeaways
# ========================================
print("=" * 70)
print("Key Takeaways: Simple Pipeline")
print("=" * 70)
print("1. ✅ Pipeline = Sequential fit/transform → Prevents data leakage")
print("2. ✅ fit() learns parameters from training data only")
print("3. ✅ predict() applies transforms (no re-fitting) → Consistency")
print("4. ✅ Single object encapsulates entire workflow → Easy serialization")
print("5. ⚠️ Manual code is error-prone (forgot to scale test data)")
print("6. 🏭 Semiconductor: Essential for multi-stage test flows (wafer → final → burn-in)")
print("=" * 70)


### 📝 What's Happening in This Code?

**Purpose:** Use **production sklearn Pipeline** with real-world semiconductor data preprocessing (StandardScaler + ColumnTransformer).

**Key Points:**
- **sklearn.pipeline.Pipeline:** Production-ready pipeline with additional features (parameter access, serialization, GridSearchCV compatibility)
- **ColumnTransformer:** Apply different preprocessing to numeric vs categorical columns (StandardScaler vs OneHotEncoder)
- **Semiconductor data:** Mix of numeric (Vdd, Idd, freq) and categorical (site_id, product_type) features
- **Model:** Random Forest for yield classification (handles non-linear relationships)
- **Validation:** Compare pipeline vs manual preprocessing to verify consistency

**Why This Matters:** Production ML requires handling heterogeneous data (numeric + categorical + text). ColumnTransformer enables clean, maintainable preprocessing. For semiconductor manufacturing, this handles site-specific categorical effects (fab location, equipment tool ID) alongside parametric test data ($10M-$50M impact from proper binning).

### 📝 Implementation

**Purpose:** Core implementation with detailed code

**Key implementation details below.**

In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
# ========================================
# Generate Semiconductor Data (Mixed Types)
# ========================================
np.random.seed(42)
n_samples = 1500
# Numeric features: Parametric test results
Vdd_min = np.random.normal(1.0, 0.1, n_samples)
Vdd_max = np.random.normal(1.2, 0.1, n_samples)
Idd_active = np.random.normal(50, 10, n_samples)
Idd_standby = np.random.normal(1, 0.5, n_samples)
freq_max = np.random.normal(2000, 200, n_samples)
temp = np.random.normal(85, 5, n_samples)
# Categorical features: Site and product type
site_ids = np.random.choice(['Site_A', 'Site_B', 'Site_C'], n_samples)
product_types = np.random.choice(['Type_X', 'Type_Y'], n_samples)
# Create DataFrame
data = pd.DataFrame({
    'Vdd_min': Vdd_min,
    'Vdd_max': Vdd_max,
    'Idd_active': Idd_active,
    'Idd_standby': Idd_standby,
    'freq_max': freq_max,
    'temp': temp,
    'site_id': site_ids,
    'product_type': product_types
})
# Generate yield labels (complex logic: numeric + categorical effects)
# Site_A has 10% yield boost, Type_Y has 5% yield boost
site_effect = (data['site_id'] == 'Site_A').astype(float) * 0.3
product_effect = (data['product_type'] == 'Type_Y').astype(float) * 0.2
y_prob = 1 / (1 + np.exp(-(
    0.8 * (data['Vdd_min'] - 1.0) / 0.1 +
    0.6 * (data['Idd_active'] - 50) / 10 +
    0.4 * (data['freq_max'] - 2000) / 200 +
    site_effect + product_effect - 0.5
)))
y = (y_prob > 0.5).astype(int)
print("=" * 80)
print("Semiconductor Yield Classification: Mixed Numeric + Categorical Data")
print("=" * 80)
print(f"Dataset: {n_samples} devices")
print(f"Numeric features: Vdd_min, Vdd_max, Idd_active, Idd_standby, freq_max, temp")
print(f"Categorical features: site_id (Site_A/B/C), product_type (Type_X/Y)")
print(f"Target: Binary yield (0 = fail, 1 = pass)")
print()
print("Sample data:")
print(data.head())
print()
print(f"Class distribution: {np.sum(y == 0)} fails ({100*np.mean(y==0):.1f}%), "
      f"{np.sum(y == 1)} passes ({100*np.mean(y==1):.1f}%)")
print()
# Split train/test
train_size = int(0.8 * n_samples)
X_train, X_test = data[:train_size], data[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
print(f"Train: {X_train.shape[0]} samples")
print(f"Test: {X_test.shape[0]} samples")
print()


### 📝 Implementation Part 2

**Purpose:** Continue implementation

**Key implementation details below.**

In [ ]:
# ========================================
# Production Pipeline with ColumnTransformer
# ========================================
print("=" * 80)
print("Building Production Pipeline: ColumnTransformer + RandomForest")
print("=" * 80)
# Define column groups
numeric_features = ['Vdd_min', 'Vdd_max', 'Idd_active', 'Idd_standby', 'freq_max', 'temp']
categorical_features = ['site_id', 'product_type']
print(f"Numeric features ({len(numeric_features)}): {numeric_features}")
print(f"Categorical features ({len(categorical_features)}): {categorical_features}")
print()
# Create preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),       # Scale numeric features
        ('cat', OneHotEncoder(drop='first'), categorical_features)  # One-hot encode categorical
    ],
    remainder='passthrough'  # Keep other columns as-is (none in this case)
)
# Create full pipeline: preprocessing + model
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42))
])
print("Pipeline steps:")
for i, (name, step) in enumerate(pipeline.steps, 1):
    print(f"  {i}. {name}: {type(step).__name__}")
print()
# Train pipeline
print("Training pipeline...")
pipeline.fit(X_train, y_train)
print("✓ Pipeline trained!")
print()
# Evaluate
y_pred_train = pipeline.predict(X_train)
y_pred_test = pipeline.predict(X_test)
train_acc = accuracy_score(y_train, y_pred_train)
test_acc = accuracy_score(y_test, y_pred_test)
print("=" * 80)
print("Pipeline Performance")
print("=" * 80)
print(f"Train Accuracy: {train_acc:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")
print()
# Classification report
print("Classification Report (Test Set):")
print(classification_report(y_test, y_pred_test, target_names=['Fail', 'Pass']))
print()
# ========================================
# Inspect ColumnTransformer Output
# ========================================
print("=" * 80)
print("ColumnTransformer Output Inspection")
print("=" * 80)
# Transform a sample with preprocessor
X_sample = X_train.iloc[:3]
X_sample_transformed = pipeline.named_steps['preprocessor'].transform(X_sample)
print("Original sample (3 devices):")
print(X_sample)
print()
print(f"Transformed sample shape: {X_sample_transformed.shape}")
print("(6 numeric scaled + 3 one-hot for site_id + 1 one-hot for product_type = 10 features)")
print()
# Get feature names after ColumnTransformer
numeric_names = numeric_features
categorical_names = pipeline.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(categorical_features)
all_feature_names = numeric_names + list(categorical_names)
print(f"Feature names after transformation ({len(all_feature_names)} features):")
for i, name in enumerate(all_feature_names, 1):
    print(f"  {i}. {name}")
print()


### 📝 Implementation Part 3

**Purpose:** Continue implementation

**Key implementation details below.**

In [ ]:
# ========================================
# Compare: Manual Preprocessing
# ========================================
print("=" * 80)
print("Validation: Manual Preprocessing (Should Match Pipeline)")
print("=" * 80)
# Manual approach
scaler_manual = StandardScaler()
encoder_manual = OneHotEncoder(drop='first')
rf_manual = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
# Preprocess numeric
X_train_num_scaled = scaler_manual.fit_transform(X_train[numeric_features])
X_test_num_scaled = scaler_manual.transform(X_test[numeric_features])
# Preprocess categorical
X_train_cat_encoded = encoder_manual.fit_transform(X_train[categorical_features]).toarray()
X_test_cat_encoded = encoder_manual.transform(X_test[categorical_features]).toarray()
# Concatenate
X_train_manual = np.hstack([X_train_num_scaled, X_train_cat_encoded])
X_test_manual = np.hstack([X_test_num_scaled, X_test_cat_encoded])
# Train
rf_manual.fit(X_train_manual, y_train)
y_pred_manual = rf_manual.predict(X_test_manual)
manual_test_acc = accuracy_score(y_test, y_pred_manual)
print(f"Manual Test Accuracy: {manual_test_acc:.4f}")
print(f"Pipeline Test Accuracy: {test_acc:.4f}")
print(f"Difference: {abs(manual_test_acc - test_acc):.6f}")
print("✓ Manual and pipeline match!" if abs(manual_test_acc - test_acc) < 1e-6 else "✗ Mismatch detected!")
print()
# ========================================
# Visualization 1: Confusion Matrix
# ========================================
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
# Pipeline
cm_pipeline = confusion_matrix(y_test, y_pred_test)
sns.heatmap(cm_pipeline, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['Fail', 'Pass'], yticklabels=['Fail', 'Pass'])
axes[0].set_title(f'Pipeline: Confusion Matrix\nAccuracy: {test_acc:.3f}', fontsize=11, weight='bold')
axes[0].set_xlabel('Predicted', fontsize=9)
axes[0].set_ylabel('Actual', fontsize=9)
# Manual
cm_manual = confusion_matrix(y_test, y_pred_manual)
sns.heatmap(cm_manual, annot=True, fmt='d', cmap='Greens', ax=axes[1],
            xticklabels=['Fail', 'Pass'], yticklabels=['Fail', 'Pass'])
axes[1].set_title(f'Manual: Confusion Matrix\nAccuracy: {manual_test_acc:.3f}', fontsize=11, weight='bold')
axes[1].set_xlabel('Predicted', fontsize=9)
axes[1].set_ylabel('Actual', fontsize=9)
plt.tight_layout()
plt.show()
print("✅ Visualization 1: Confusion matrices (pipeline vs manual)")
print()
# ========================================
# Visualization 2: Feature Importance from Pipeline
# ========================================
# Extract feature importance from pipeline
feature_importance = pipeline.named_steps['classifier'].feature_importances_
# Sort by importance
importance_df = pd.DataFrame({
    'Feature': all_feature_names,
    'Importance': feature_importance
}).sort_values('Importance', ascending=False)
plt.figure(figsize=(10, 6))
plt.barh(importance_df['Feature'], importance_df['Importance'], color='steelblue', edgecolor='black')
plt.xlabel('Feature Importance', fontsize=11, weight='bold')
plt.ylabel('Feature', fontsize=11, weight='bold')
plt.title('Random Forest Feature Importance (from Pipeline)', fontsize=12, weight='bold')
plt.gca().invert_yaxis()
plt.grid(alpha=0.3, axis='x')
plt.tight_layout()
plt.show()
print("✅ Visualization 2: Feature importance from pipeline model")
print()
print("Top 5 features:")
print(importance_df.head())
print()


### 📝 Implementation Part 4

**Purpose:** Continue implementation

**Key implementation details below.**

In [ ]:
# ========================================
# Visualization 3: Site Effect Comparison
# ========================================
# Group test predictions by site
site_results = pd.DataFrame({
    'site_id': X_test['site_id'],
    'actual': y_test,
    'predicted': y_pred_test
})
site_accuracy = site_results.groupby('site_id').apply(
    lambda g: accuracy_score(g['actual'], g['predicted'])
).sort_values(ascending=False)
plt.figure(figsize=(8, 5))
plt.bar(site_accuracy.index, site_accuracy.values, color=['#2ecc71', '#3498db', '#e74c3c'], edgecolor='black', linewidth=1.5)
plt.ylabel('Accuracy', fontsize=11, weight='bold')
plt.xlabel('Site ID', fontsize=11, weight='bold')
plt.title('Model Accuracy by Manufacturing Site', fontsize=12, weight='bold')
plt.ylim([0, 1])
plt.grid(alpha=0.3, axis='y')
plt.tight_layout()
plt.show()
print("✅ Visualization 3: Accuracy by site (categorical feature impact)")
print()
print("Site-wise accuracy:")
print(site_accuracy)
print()
# ========================================
# Key Takeaways
# ========================================
print("=" * 80)
print("Key Takeaways: Production Pipeline with ColumnTransformer")
print("=" * 80)
print("1. ✅ ColumnTransformer: Handle heterogeneous data (numeric + categorical)")
print("2. ✅ Pipeline encapsulates entire workflow: Clean, reproducible, serializable")
print("3. ✅ One-hot encoding: Categorical features → binary features (drop='first' avoids multicollinearity)")
print("4. ✅ Feature importance: Extract from pipeline.named_steps['classifier']")
print("5. ✅ Validation: Manual preprocessing matches pipeline (proves correctness)")
print("6. 🏭 Semiconductor: Site/product categorical effects captured ($10M-$50M binning impact)")
print("=" * 80)


## 🔧 Custom Transformers: Domain-Specific Preprocessing

### **When to Write Custom Transformers**

While sklearn provides many built-in transformers (`StandardScaler`, `OneHotEncoder`, `PCA`), real-world applications often require **domain-specific preprocessing** that doesn't exist in standard libraries.

**Common scenarios requiring custom transformers:**

1. **Semiconductor Testing:**
   - Spatial detrending for wafer maps (remove edge-die effects)
   - Multi-site normalization (harmonize data from different fabs)
   - Temporal filtering (remove test equipment drift over time)
   - Outlier capping based on spec limits (not statistical outliers)

2. **Time Series:**
   - Rolling window statistics (mean, std, quantiles over last N periods)
   - Lag features (previous values as predictors)
   - Seasonal decomposition (trend + seasonal + residual)

3. **Text/NLP:**
   - Custom tokenization (domain-specific abbreviations, product codes)
   - Feature extraction from structured text (part numbers, serial numbers)

4. **Business Logic:**
   - Conditional transformations (different logic for different customer segments)
   - Domain constraints (physical limits, regulatory requirements)

---

### **Custom Transformer Template**

**Blueprint:** Inherit from `BaseEstimator` and `TransformerMixin`

```python
from sklearn.base import BaseEstimator, TransformerMixin

class MyCustomTransformer(BaseEstimator, TransformerMixin):
    """
    Template for custom transformer.
    
    Parameters:
    -----------
    hyperparam1 : type
        Description of hyperparameter
    """
    
    def __init__(self, hyperparam1=default_value):
        # Store hyperparameters (no data-dependent logic!)
        self.hyperparam1 = hyperparam1
    
    def fit(self, X, y=None):
        """
        Learn parameters from training data.
        
        Parameters:
        -----------
        X : array-like, shape (n_samples, n_features)
            Training data
        y : array-like, shape (n_samples,), optional
            Target values (for supervised transformers)
        
        Returns:
        --------
        self : object
            Returns self for method chaining
        """
        # Compute statistics from training data
        # Store learned parameters with trailing underscore
        self.learned_param_ = compute_statistic(X)
        
        return self
    
    def transform(self, X):
        """
        Apply transformation using learned parameters.
        
        Parameters:
        -----------
        X : array-like, shape (n_samples, n_features)
            Data to transform
        
        Returns:
        --------
        X_transformed : array-like
            Transformed data
        """
        # Check if fit() was called
        if not hasattr(self, 'learned_param_'):
            raise ValueError("Transformer not fitted. Call fit() first.")
        
        # Apply transformation using self.learned_param_
        X_transformed = apply_transformation(X, self.learned_param_)
        
        return X_transformed
    
    def fit_transform(self, X, y=None):
        """
        Fit and transform in one step (provided by TransformerMixin).
        """
        return self.fit(X, y).transform(X)
```

---

### **Key Rules for Custom Transformers**

| **Rule** | **Explanation** | **Example** |
|----------|----------------|-------------|
| **1. Inherit from BaseEstimator + TransformerMixin** | Provides `fit_transform()`, `get_params()`, `set_params()` | `class MyTransformer(BaseEstimator, TransformerMixin):` |
| **2. `__init__` only stores hyperparameters** | No data-dependent logic in constructor | `self.threshold = threshold` ✅<br>`self.mean = X.mean()` ❌ |
| **3. Learned parameters end with `_`** | Convention: trailing underscore = learned from data | `self.mean_`, `self.std_`, `self.components_` |
| **4. `fit()` returns `self`** | Enables method chaining | `return self` |
| **5. `transform()` checks if fitted** | Prevent errors from unfitted transformer | `if not hasattr(self, 'mean_'): raise ValueError(...)` |
| **6. Stateless transforms use `FunctionTransformer`** | For simple functions (no learned parameters) | `FunctionTransformer(np.log)` |

---

### **Example 1: Wafer Spatial Detrending (Semiconductor)**

**Problem:** Edge dies have lower yield than center dies (spatial bias)

**Mathematical formulation:**

$$
Y_{i,j}^{\text{detrended}} = Y_{i,j} - \bar{Y}_{\text{wafer}}
$$

Where:
- $Y_{i,j}$: Yield at die position $(x_i, y_j)$
- $\bar{Y}_{\text{wafer}}$: Mean yield across all dies on the wafer

**Custom transformer:**

```python
class WaferSpatialDetrending(BaseEstimator, TransformerMixin):
    def __init__(self, group_col='wafer_id', value_col='yield'):
        self.group_col = group_col
        self.value_col = value_col
    
    def fit(self, X, y=None):
        # Learn wafer-level means from training data
        self.wafer_means_ = X.groupby(self.group_col)[self.value_col].mean()
        return self
    
    def transform(self, X):
        X = X.copy()
        # Subtract wafer-level mean from each die
        X[f'{self.value_col}_detrended'] = (
            X[self.value_col] - X[self.group_col].map(self.wafer_means_)
        )
        return X
```

---

### **Example 2: Rolling Window Features (Time Series)**

**Problem:** Capture temporal patterns (moving average, volatility)

**Mathematical formulation:**

$$
\text{MA}_t = \frac{1}{w} \sum_{i=t-w+1}^{t} X_i
$$

$$
\text{Vol}_t = \sqrt{\frac{1}{w} \sum_{i=t-w+1}^{t} (X_i - \text{MA}_t)^2}
$$

**Custom transformer:**

```python
class RollingWindowFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, window_size=7, features=['mean', 'std']):
        self.window_size = window_size
        self.features = features
    
    def fit(self, X, y=None):
        # No parameters to learn (stateless for rolling windows)
        return self
    
    def transform(self, X):
        X = X.copy()
        for col in X.columns:
            if 'mean' in self.features:
                X[f'{col}_rolling_mean'] = X[col].rolling(self.window_size).mean()
            if 'std' in self.features:
                X[f'{col}_rolling_std'] = X[col].rolling(self.window_size).std()
        return X.fillna(0)  # Handle NaN at start
```

---

### **Example 3: Outlier Capping (Business Logic)**

**Problem:** Cap extreme values at specified percentiles (not removal)

**Mathematical formulation:**

$$
X_{\text{capped}} = \begin{cases}
q_{\text{lower}} & \text{if } X < q_{\text{lower}} \\
q_{\text{upper}} & \text{if } X > q_{\text{upper}} \\
X & \text{otherwise}
\end{cases}
$$

**Custom transformer:**

```python
class OutlierCapper(BaseEstimator, TransformerMixin):
    def __init__(self, lower_percentile=5, upper_percentile=95):
        self.lower_percentile = lower_percentile
        self.upper_percentile = upper_percentile
    
    def fit(self, X, y=None):
        # Learn percentile bounds from training data
        self.lower_bounds_ = np.percentile(X, self.lower_percentile, axis=0)
        self.upper_bounds_ = np.percentile(X, self.upper_percentile, axis=0)
        return self
    
    def transform(self, X):
        X_capped = np.clip(X, self.lower_bounds_, self.upper_bounds_)
        return X_capped
```

---

### **Integration with Pipeline**

**Usage pattern:**

```python
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('detrending', WaferSpatialDetrending(group_col='wafer_id')),
    ('capping', OutlierCapper(lower_percentile=5, upper_percentile=95)),
    ('scaler', StandardScaler()),
    ('model', RandomForestClassifier())
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
```

**Benefit:** Custom domain logic is encapsulated, reusable, and testable.

---

### **Testing Custom Transformers**

**Unit test pattern:**

```python
def test_wafer_detrending():
    # Create test data
    data = pd.DataFrame({
        'wafer_id': [1, 1, 2, 2],
        'yield': [0.9, 0.8, 0.7, 0.6]
    })
    
    # Fit transformer
    transformer = WaferSpatialDetrending()
    transformer.fit(data)
    
    # Check learned parameters
    assert transformer.wafer_means_[1] == 0.85  # (0.9 + 0.8) / 2
    assert transformer.wafer_means_[2] == 0.65  # (0.7 + 0.6) / 2
    
    # Transform
    data_transformed = transformer.transform(data)
    
    # Check detrended values
    assert data_transformed['yield_detrended'].iloc[0] == 0.05  # 0.9 - 0.85
    assert data_transformed['yield_detrended'].iloc[2] == 0.05  # 0.7 - 0.65
    
    print("✓ WaferSpatialDetrending test passed!")
```

---

### **Common Pitfalls and Solutions**

| **Pitfall** | **Problem** | **Solution** |
|-------------|-------------|--------------|
| **Data leakage in `__init__`** | Computing statistics in constructor | Move to `fit()` method |
| **Forget trailing `_`** | Learned parameters without underscore | Follow sklearn convention: `self.mean_` |
| **Forget to return `self`** | `fit()` doesn't return self → breaks chaining | Always `return self` in `fit()` |
| **Modify X in-place** | Original data is changed | Use `X = X.copy()` at start of `transform()` |
| **No fit check** | Transform called before fit → crash | Add `if not hasattr(self, 'param_'): raise ValueError(...)` |
| **Wrong shape** | Return shape doesn't match input | Ensure `X_out.shape[0] == X.shape[0]` |

---

### **Next: Implementing Custom Transformers**

We'll build:
1. **SemiconductorFeatureEngineer** - Domain-specific features for wafer test data
2. **Integration with Pipeline** - Seamless fit/transform flow
3. **Validation** - Unit tests and visual inspection

Let's code! 🛠️

### 📝 What's Happening in This Code?

**Purpose:** Implement **custom transformer for semiconductor-specific feature engineering** (spatial detrending, parametric ratios, interaction features).

**Key Points:**
- **SemiconductorFeatureEngineer:** Custom transformer inheriting from BaseEstimator + TransformerMixin
- **fit() method:** Learns wafer-level statistics from training data (prevents data leakage)
- **transform() method:** Creates domain-specific features (Vdd/Idd ratios, spatial detrending, power-frequency interactions)
- **Pipeline integration:** Seamlessly fits into sklearn pipelines with other transformers
- **Unit testing:** Validate learned parameters and output shapes to ensure correctness

**Why This Matters:** Custom transformers enable domain expertise to be encoded in reusable, testable components. For semiconductor testing, spatial effects and parametric relationships are critical for accurate yield prediction ($10M-$50M annual impact from proper feature engineering).

### 📝 Implementation

**Purpose:** Core implementation with detailed code

**Key implementation details below.**

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
# ========================================
# Custom Transformer: SemiconductorFeatureEngineer
# ========================================
class SemiconductorFeatureEngineer(BaseEstimator, TransformerMixin):
    """
    Custom transformer for semiconductor-specific feature engineering.
    
    Creates domain-specific features:
    1. Spatial detrending: Remove wafer-level bias
    2. Parametric ratios: Vdd/Idd, Power/Freq relationships
    3. Interaction features: Voltage * Current, Freq * Power
    
    Parameters:
    -----------
    spatial_detrending : bool, default=True
        Whether to apply spatial detrending (remove wafer-level mean)
    create_ratios : bool, default=True
        Whether to create parametric ratio features
    create_interactions : bool, default=True
        Whether to create interaction features
    
    Attributes (learned during fit):
    ---------------------------------
    wafer_means_ : dict
        Mean values for each wafer (for spatial detrending)
    feature_names_ : list
        Names of all output features
    """
    
    def __init__(self, spatial_detrending=True, create_ratios=True, create_interactions=True):
        self.spatial_detrending = spatial_detrending
        self.create_ratios = create_ratios
        self.create_interactions = create_interactions
    
    def fit(self, X, y=None):
        """
        Learn wafer-level statistics from training data.
        
        Parameters:
        -----------
        X : pd.DataFrame
            Training data with columns: wafer_id, Vdd_min, Vdd_max, Idd_active, etc.
        y : array-like, optional
            Target values (not used for feature engineering)
        
        Returns:
        --------
        self : object
            Returns self for method chaining
        """
        if self.spatial_detrending:
            # Learn wafer-level means from training data
            if 'wafer_id' not in X.columns:
                raise ValueError("X must contain 'wafer_id' column for spatial detrending")
            
            # Compute mean Vdd_min for each wafer (can extend to other features)
            self.wafer_means_ = X.groupby('wafer_id')['Vdd_min'].mean().to_dict()
            print(f"[SemiconductorFeatureEngineer] Learned {len(self.wafer_means_)} wafer-level means")
        
        # Store feature names for reference
        self.feature_names_ = self._get_feature_names(X)
        
        return self
    
    def transform(self, X):
        """
        Apply semiconductor-specific feature engineering.
        
        Parameters:
        -----------
        X : pd.DataFrame
            Data to transform
        
        Returns:
        --------
        X_transformed : pd.DataFrame
            Data with additional engineered features
        """
        # Check if fitted
        if self.spatial_detrending and not hasattr(self, 'wafer_means_'):
            raise ValueError("Transformer not fitted. Call fit() before transform().")
        
        X = X.copy()
        
        # Feature 1: Spatial detrending
        if self.spatial_detrending:
            # Map wafer_id to mean, handle unseen wafers with global mean
            global_mean = np.mean(list(self.wafer_means_.values()))
            X['Vdd_min_detrended'] = X['Vdd_min'] - X['wafer_id'].map(self.wafer_means_).fillna(global_mean)
        
        # Feature 2: Parametric ratios
        if self.create_ratios:
            # Voltage ratio (Vdd_max / Vdd_min): Higher ratio = wider operating margin
            X['Vdd_ratio'] = X['Vdd_max'] / (X['Vdd_min'] + 1e-6)  # Avoid division by zero
            
            # Current efficiency (Idd_active / Idd_standby): Lower = better power management
            X['Idd_efficiency'] = X['Idd_active'] / (X['Idd_standby'] + 1e-6)
            
            # Power density (Idd_active / freq_max): Lower = more efficient at high frequency
            X['power_per_freq'] = X['Idd_active'] / (X['freq_max'] + 1e-6)
        
        # Feature 3: Interaction features
        if self.create_interactions:
            # Power = Voltage * Current
            X['power_estimate'] = X['Vdd_max'] * X['Idd_active']
            
            # High-frequency power stress = freq * power
            X['freq_power_stress'] = X['freq_max'] * X['power_estimate']
            
            # Temperature-adjusted leakage = temp * Idd_standby
            if 'temp' in X.columns:
                X['temp_leakage'] = X['temp'] * X['Idd_standby']
        
        return X
    
    def _get_feature_names(self, X):
        """Helper to get list of all output feature names."""
        names = list(X.columns)
        
        if self.spatial_detrending:
            names.append('Vdd_min_detrended')
        
        if self.create_ratios:
            names.extend(['Vdd_ratio', 'Idd_efficiency', 'power_per_freq'])
        
        if self.create_interactions:
            names.extend(['power_estimate', 'freq_power_stress'])
            if 'temp' in X.columns:
                names.append('temp_leakage')
        
        return names


### 📝 Implementation Part 2

**Purpose:** Continue implementation

**Key implementation details below.**

In [ ]:
# ========================================
# Generate Test Data with Wafer IDs
# ========================================
np.random.seed(42)
n_wafers = 10
dies_per_wafer = 100
n_samples = n_wafers * dies_per_wafer
# Generate wafer IDs
wafer_ids = np.repeat(range(n_wafers), dies_per_wafer)
# Generate parametric test data
# Each wafer has different mean (spatial effect)
wafer_offsets = np.random.randn(n_wafers) * 0.1  # Wafer-level bias
Vdd_min = 1.0 + wafer_offsets[wafer_ids] + np.random.randn(n_samples) * 0.05
Vdd_max = 1.2 + wafer_offsets[wafer_ids] + np.random.randn(n_samples) * 0.05
Idd_active = 50 + np.random.randn(n_samples) * 5
Idd_standby = 1 + np.random.randn(n_samples) * 0.2
freq_max = 2000 + np.random.randn(n_samples) * 100
temp = 85 + np.random.randn(n_samples) * 5
data = pd.DataFrame({
    'wafer_id': wafer_ids,
    'Vdd_min': Vdd_min,
    'Vdd_max': Vdd_max,
    'Idd_active': Idd_active,
    'Idd_standby': Idd_standby,
    'freq_max': freq_max,
    'temp': temp
})
# Generate yield labels (spatial + parametric effects)
y_prob = 1 / (1 + np.exp(-(
    5 * (data['Vdd_min'] - 1.0) +
    0.01 * (data['Idd_active'] - 50) +
    0.001 * (data['freq_max'] - 2000) -
    0.5
)))
y = (y_prob > 0.5).astype(int)
print("=" * 80)
print("Semiconductor Data with Spatial Effects")
print("=" * 80)
print(f"Dataset: {n_samples} dies from {n_wafers} wafers")
print(f"Features: wafer_id, Vdd_min, Vdd_max, Idd_active, Idd_standby, freq_max, temp")
print()
print("Sample data:")
print(data.head(10))
print()
# Wafer-level statistics
wafer_stats = data.groupby('wafer_id').agg({
    'Vdd_min': 'mean',
    'Vdd_max': 'mean'
}).round(3)
print("Wafer-level means (showing spatial variation):")
print(wafer_stats.head())
print()
# Split train/test
train_size = int(0.8 * n_samples)
X_train, X_test = data[:train_size], data[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
# ========================================
# Apply Custom Transformer
# ========================================
print("=" * 80)
print("Applying SemiconductorFeatureEngineer")
print("=" * 80)
transformer = SemiconductorFeatureEngineer(
    spatial_detrending=True,
    create_ratios=True,
    create_interactions=True
)
# Fit on training data
transformer.fit(X_train)
print()
# Transform training and test data
X_train_transformed = transformer.transform(X_train)
X_test_transformed = transformer.transform(X_test)
print(f"Original shape: {X_train.shape}")
print(f"Transformed shape: {X_train_transformed.shape}")
print(f"New features added: {X_train_transformed.shape[1] - X_train.shape[1]}")
print()
print("New feature names:")
for i, name in enumerate(transformer.feature_names_[X_train.shape[1]:], 1):
    print(f"  {i}. {name}")
print()
print("Sample transformed data (first 3 dies):")
print(X_train_transformed.head(3)[['Vdd_min', 'Vdd_min_detrended', 'Vdd_ratio', 
                                     'power_estimate', 'freq_power_stress']])
print()


### 📝 Implementation Part 3

**Purpose:** Continue implementation

**Key implementation details below.**

In [ ]:
# ========================================
# Unit Test: Validate Learned Parameters
# ========================================
print("=" * 80)
print("Unit Test: Validate Learned Parameters")
print("=" * 80)
# Test 1: Check wafer means
manual_wafer_means = X_train.groupby('wafer_id')['Vdd_min'].mean()
learned_wafer_means = pd.Series(transformer.wafer_means_)
difference = (manual_wafer_means - learned_wafer_means).abs().max()
print(f"Test 1: Wafer means match manual calculation")
print(f"  Max difference: {difference:.10f}")
print(f"  Status: {'✓ PASS' if difference < 1e-10 else '✗ FAIL'}")
print()
# Test 2: Check detrended values
wafer_0_mean = transformer.wafer_means_[0]
wafer_0_dies = X_train_transformed[X_train_transformed['wafer_id'] == 0]
expected_detrended = wafer_0_dies['Vdd_min'] - wafer_0_mean
actual_detrended = wafer_0_dies['Vdd_min_detrended']
detrend_diff = (expected_detrended - actual_detrended).abs().max()
print(f"Test 2: Detrended values correct for wafer 0")
print(f"  Max difference: {detrend_diff:.10f}")
print(f"  Status: {'✓ PASS' if detrend_diff < 1e-10 else '✗ FAIL'}")
print()
# Test 3: Check ratio features
expected_ratio = X_train_transformed['Vdd_max'] / (X_train_transformed['Vdd_min'] + 1e-6)
actual_ratio = X_train_transformed['Vdd_ratio']
ratio_diff = (expected_ratio - actual_ratio).abs().max()
print(f"Test 3: Vdd_ratio calculated correctly")
print(f"  Max difference: {ratio_diff:.10f}")
print(f"  Status: {'✓ PASS' if ratio_diff < 1e-10 else '✗ FAIL'}")
print()
# ========================================
# Integrate with Full Pipeline
# ========================================
print("=" * 80)
print("Full Pipeline: Custom Transformer + StandardScaler + Classifier")
print("=" * 80)
# Drop non-numeric columns for model
numeric_cols = ['Vdd_min', 'Vdd_max', 'Idd_active', 'Idd_standby', 'freq_max', 'temp',
                'Vdd_min_detrended', 'Vdd_ratio', 'Idd_efficiency', 'power_per_freq',
                'power_estimate', 'freq_power_stress', 'temp_leakage']
from sklearn.pipeline import Pipeline, FunctionTransformer
# Helper function to select numeric columns
def select_numeric(X):
    return X[numeric_cols]
# Build pipeline
pipeline_with_custom = Pipeline([
    ('feature_engineer', SemiconductorFeatureEngineer()),
    ('select_numeric', FunctionTransformer(select_numeric)),
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(max_iter=1000))
])
# Train
pipeline_with_custom.fit(X_train, y_train)
# Evaluate
y_pred_train = pipeline_with_custom.predict(X_train)
y_pred_test = pipeline_with_custom.predict(X_test)
train_acc = accuracy_score(y_train, y_pred_train)
test_acc = accuracy_score(y_test, y_pred_test)
print(f"Train Accuracy: {train_acc:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")
print()


### 📝 Implementation Part 4

**Purpose:** Continue implementation

**Key implementation details below.**

In [ ]:
# ========================================
# Compare: Pipeline with vs without Custom Features
# ========================================
print("=" * 80)
print("Comparison: With vs Without Custom Features")
print("=" * 80)
# Pipeline WITHOUT custom features
basic_pipeline = Pipeline([
    ('select_numeric', FunctionTransformer(lambda X: X[['Vdd_min', 'Vdd_max', 'Idd_active', 
                                                         'Idd_standby', 'freq_max', 'temp']])),
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(max_iter=1000))
])
basic_pipeline.fit(X_train, y_train)
basic_test_acc = basic_pipeline.score(X_test, y_test)
print(f"Without custom features: {basic_test_acc:.4f}")
print(f"With custom features:    {test_acc:.4f}")
print(f"Improvement:             {test_acc - basic_test_acc:.4f} ({100*(test_acc - basic_test_acc)/basic_test_acc:.1f}%)")
print()
# ========================================
# Visualization: Feature Importance
# ========================================
# Retrain with Random Forest to get feature importance
from sklearn.ensemble import RandomForestClassifier
pipeline_rf = Pipeline([
    ('feature_engineer', SemiconductorFeatureEngineer()),
    ('select_numeric', FunctionTransformer(select_numeric)),
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42))
])
pipeline_rf.fit(X_train, y_train)
# Get feature importance
feature_importance = pipeline_rf.named_steps['classifier'].feature_importances_
importance_df = pd.DataFrame({
    'Feature': numeric_cols,
    'Importance': feature_importance
}).sort_values('Importance', ascending=False)
plt.figure(figsize=(10, 6))
plt.barh(importance_df['Feature'][:10], importance_df['Importance'][:10], 
         color='steelblue', edgecolor='black')
plt.xlabel('Feature Importance', fontsize=11, weight='bold')
plt.ylabel('Feature', fontsize=11, weight='bold')
plt.title('Top 10 Features (Including Custom Engineered Features)', fontsize=12, weight='bold')
plt.gca().invert_yaxis()
plt.grid(alpha=0.3, axis='x')
plt.tight_layout()
plt.show()
print("✅ Visualization: Feature importance with custom features")
print()
# Highlight custom features
custom_features = ['Vdd_min_detrended', 'Vdd_ratio', 'Idd_efficiency', 'power_per_freq',
                   'power_estimate', 'freq_power_stress', 'temp_leakage']
custom_importance = importance_df[importance_df['Feature'].isin(custom_features)]
print("Custom feature importance:")
print(custom_importance)
print()


### 📝 Implementation Part 5

**Purpose:** Continue implementation

**Key implementation details below.**

In [ ]:
# ========================================
# Key Takeaways
# ========================================
print("=" * 80)
print("Key Takeaways: Custom Transformers")
print("=" * 80)
print("1. ✅ Custom transformer = Domain expertise + Reusability + Testability")
print("2. ✅ fit() learns from training data only → No data leakage")
print("3. ✅ transform() applies consistent logic to train/test/production")
print("4. ✅ Seamless integration with sklearn Pipeline")
print("5. ✅ Unit tests validate correctness (wafer means, detrended values, ratios)")
print("6. 🏭 Semiconductor: Spatial detrending + parametric ratios improve accuracy")
print("7. 📈 Custom features: 7 new features → ~5-10% accuracy improvement")
print("=" * 80)


## 🔀 FeatureUnion: Parallel Feature Extraction

### **The Problem: Multiple Feature Representations**

Often, we want to combine **multiple feature extraction strategies** in parallel:

1. **Dimensionality reduction** (PCA) - Capture linear patterns
2. **Feature selection** (SelectKBest) - Keep most predictive features
3. **Polynomial features** - Capture non-linear interactions
4. **Custom domain features** - Domain-specific engineering

**Challenge:** How to combine these without sequential bottlenecks?

---

### **Sequential Approach (Suboptimal)**

```python
# Option 1: PCA only
X_pca = PCA(n_components=50).fit_transform(X)

# Option 2: SelectKBest only
X_selected = SelectKBest(k=30).fit_transform(X)

# Can't easily combine both!
```

**Problem:** Must choose ONE strategy, losing information from the other.

---

### **FeatureUnion Solution: Parallel Combination**

```python
from sklearn.pipeline import FeatureUnion

feature_union = FeatureUnion([
    ('pca', PCA(n_components=50)),           # Extract 50 PCA features
    ('select', SelectKBest(k=30))            # Extract 30 best features
])

X_combined = feature_union.fit_transform(X)  # Shape: (n_samples, 50 + 30 = 80)
```

**Mathematical structure:**

$$
\text{FeatureUnion}(X) = [f_1(X) \mid f_2(X) \mid \cdots \mid f_k(X)]
$$

Where $\mid$ denotes horizontal concatenation.

---

### **FeatureUnion Architecture**

```mermaid
graph TD
    A[Raw Features<br/>X: n × p] --> B[Transformer 1<br/>PCA]
    A --> C[Transformer 2<br/>SelectKBest]
    A --> D[Transformer 3<br/>CustomTransformer]
    
    B --> E[Features 1<br/>n × p1]
    C --> F[Features 2<br/>n × p2]
    D --> G[Features 3<br/>n × p3]
    
    E --> H[Concatenate<br/>n × p1+p2+p3]
    F --> H
    G --> H
    
    H --> I[Combined Features<br/>X_combined]
    
    style A fill:#ffe6e6
    style B fill:#e6f3ff
    style C fill:#e6f3ff
    style D fill:#e6f3ff
    style H fill:#fff3e6
    style I fill:#90EE90
```

**Key insight:** Transformers run **in parallel**, not sequentially.

---

### **FeatureUnion vs Pipeline**

| **Aspect** | **Pipeline** | **FeatureUnion** |
|------------|--------------|------------------|
| **Structure** | Sequential: $f_3(f_2(f_1(X)))$ | Parallel: $[f_1(X) \mid f_2(X) \mid f_3(X)]$ |
| **Input to each step** | Output of previous step | Original input $X$ |
| **Output shape** | Depends on last transform | Sum of all transformer outputs |
| **Use case** | Preprocessing sequence | Combine feature representations |
| **Example** | Scaler → PCA → Model | PCA features + SelectKBest features |

---

### **Mathematical Benefits**

**1. Capture different aspects of data:**

- **PCA:** Linear combinations maximizing variance
  $$
  X_{\text{PCA}} = X \cdot W_{\text{PCA}} \quad \text{(shape: } n \times k_1\text{)}
  $$

- **SelectKBest:** Original features with highest correlation to target
  $$
  X_{\text{selected}} = X[:, \text{indices}] \quad \text{(shape: } n \times k_2\text{)}
  $$

- **Combined:**
  $$
  X_{\text{union}} = [X_{\text{PCA}} \mid X_{\text{selected}}] \quad \text{(shape: } n \times (k_1 + k_2)\text{)}
  $$

**2. Regularization through diversity:**
- PCA: Captures global variance patterns
- SelectKBest: Captures local target correlation
- Union: Model learns from both perspectives → better generalization

---

### **Common Use Cases**

**1. Text Processing:**

```python
text_features = FeatureUnion([
    ('tfidf', TfidfVectorizer(max_features=1000)),      # Bag of words
    ('char_ngrams', TfidfVectorizer(analyzer='char', ngram_range=(2,4), max_features=500))  # Character n-grams
])
# Output: 1000 word features + 500 character n-gram features = 1500 features
```

**2. Image Processing:**

```python
image_features = FeatureUnion([
    ('color_hist', ColorHistogramExtractor()),     # Color distribution
    ('edge_features', EdgeDetector()),             # Edge patterns
    ('texture', TextureAnalyzer())                 # Texture statistics
])
```

**3. Semiconductor Testing:**

```python
semiconductor_features = FeatureUnion([
    ('parametric', SemiconductorFeatureEngineer()),  # Domain-specific ratios
    ('spatial', WaferSpatialFeatures()),             # Spatial statistics
    ('temporal', TestTimeFeatures())                 # Temporal patterns
])
```

---

### **FeatureUnion + Pipeline Integration**

**Combining Pipeline (sequential) and FeatureUnion (parallel):**

```python
from sklearn.pipeline import Pipeline, FeatureUnion

# Create feature union
feature_engineering = FeatureUnion([
    ('pca_branch', Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=50))
    ])),
    ('select_branch', Pipeline([
        ('scaler', StandardScaler()),
        ('select', SelectKBest(k=30))
    ]))
])

# Integrate into full pipeline
full_pipeline = Pipeline([
    ('feature_union', feature_engineering),
    ('classifier', RandomForestClassifier())
])
```

**Execution flow:**
1. `feature_engineering` runs two branches in parallel
2. Each branch has its own scaler (independent normalization)
3. Outputs concatenated: 50 + 30 = 80 features
4. Classifier trained on combined features

---

### **Weighting Transformers**

**Problem:** Some feature sets more important than others

**Solution:** Use `transformer_weights` parameter

```python
feature_union = FeatureUnion([
    ('pca', PCA(n_components=50)),
    ('select', SelectKBest(k=30))
], transformer_weights={
    'pca': 2.0,      # Weight PCA features 2x
    'select': 1.0    # Weight selected features 1x
})
```

**Mathematical effect:**

$$
X_{\text{weighted}} = [2.0 \cdot X_{\text{PCA}} \mid 1.0 \cdot X_{\text{selected}}]
$$

**Use case:** When domain knowledge suggests one representation is more reliable.

---

### **Caching with FeatureUnion**

**Problem:** Expensive feature extraction (e.g., deep learning embeddings)

**Solution:** Cache intermediate results

```python
from sklearn.pipeline import Pipeline
from joblib import Memory

memory = Memory(location='/tmp/cache', verbose=0)

feature_union = FeatureUnion([
    ('expensive_features', ExpensiveTransformer()),
    ('cheap_features', CheapTransformer())
], memory=memory)
```

**Benefit:** During GridSearchCV, expensive features computed once, reused 10x (for 10-fold CV).

---

### **FeatureUnion Pitfalls**

| **Pitfall** | **Problem** | **Solution** |
|-------------|-------------|--------------|
| **Shape mismatch** | Transformers return different n_samples | Ensure all transformers preserve sample order |
| **Feature scaling** | Some branches scaled, others not | Include scaling in each branch independently |
| **Redundant features** | PCA + Original features → high correlation | Use regularization (L1, L2) or dimensionality reduction |
| **Memory explosion** | Too many features (10K + 5K + 3K = 18K) | Feature selection after union, or reduce component counts |

---

### **Next: Hands-On FeatureUnion Implementation**

We'll build:
1. **FeatureUnion with PCA + SelectKBest** - Combine linear + selective features
2. **Integration with custom transformers** - Domain features + statistical features
3. **Performance comparison** - Union vs individual strategies

Let's code! 🚀

### 📝 What's Happening in This Code?

**Purpose:** Demonstrate **FeatureUnion for combining PCA + SelectKBest features** in parallel, then integrate with full pipeline.

**Key Points:**
- **FeatureUnion:** Runs PCA (50 components) and SelectKBest (30 features) in parallel, concatenates results
- **Parallel branches:** Each branch has independent StandardScaler (different normalization strategies)
- **Combined features:** 50 PCA + 30 selected = 80 total features for model training
- **Performance comparison:** Union vs PCA-only vs SelectKBest-only to quantify improvement
- **GridSearchCV integration:** Tune FeatureUnion hyperparameters (n_components, k) alongside model hyperparameters

**Why This Matters:** Real-world data has multiple signal types (linear patterns, non-linear relationships, domain features). FeatureUnion captures diverse representations, improving model robustness. For semiconductor testing, combining spatial PCA features with selected parametric features yields 5-15% accuracy gains ($5M-$20M annual value).

### 📝 Implementation

**Purpose:** Core implementation with detailed code

**Key implementation details below.**

In [ ]:
from sklearn.pipeline import FeatureUnion
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import GridSearchCV
# ========================================
# Generate Semiconductor Data
# ========================================
np.random.seed(42)
n_samples = 1200
n_features = 80  # Large feature space (typical for semiconductor testing)
# Generate features with different characteristics:
# - Features 0-19: Highly informative (linear signal)
# - Features 20-39: Moderately informative (non-linear signal)
# - Features 40-79: Noise (low signal)
# Signal features
X_signal_linear = np.random.randn(n_samples, 20)
X_signal_nonlinear = np.random.randn(n_samples, 20) ** 2
X_noise = np.random.randn(n_samples, 40) * 0.5
X = np.hstack([X_signal_linear, X_signal_nonlinear, X_noise])
# Generate target (depends on linear + non-linear features)
y_prob = 1 / (1 + np.exp(-(
    np.sum(X[:, :10], axis=1) * 0.3 +           # Linear features 0-9
    np.sum(X[:, 20:25] ** 2, axis=1) * 0.1 -    # Non-linear features 20-24
    2.0
)))
y = (y_prob > 0.5).astype(int)
print("=" * 80)
print("High-Dimensional Semiconductor Data")
print("=" * 80)
print(f"Dataset: {n_samples} samples, {n_features} features")
print(f"Feature types:")
print(f"  - Features 0-19:  Linear signal (high importance)")
print(f"  - Features 20-39: Non-linear signal (moderate importance)")
print(f"  - Features 40-79: Noise (low importance)")
print(f"Target: Binary classification (0 = fail, 1 = pass)")
print(f"Class distribution: {np.sum(y==0)} fails, {np.sum(y==1)} passes")
print()
# Split train/test
train_size = int(0.8 * n_samples)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
# ========================================
# Strategy 1: PCA Only
# ========================================
print("=" * 80)
print("Strategy 1: PCA Only (50 components)")
print("=" * 80)
pipeline_pca = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=50)),
    ('classifier', LogisticRegression(max_iter=1000))
])
pipeline_pca.fit(X_train, y_train)
pca_test_acc = pipeline_pca.score(X_test, y_test)
print(f"Test Accuracy (PCA only): {pca_test_acc:.4f}")
print()


### 📝 Implementation Part 2

**Purpose:** Continue implementation

**Key implementation details below.**

In [ ]:
# ========================================
# Strategy 2: SelectKBest Only
# ========================================
print("=" * 80)
print("Strategy 2: SelectKBest Only (30 features)")
print("=" * 80)
pipeline_select = Pipeline([
    ('scaler', StandardScaler()),
    ('select', SelectKBest(f_classif, k=30)),
    ('classifier', LogisticRegression(max_iter=1000))
])
pipeline_select.fit(X_train, y_train)
select_test_acc = pipeline_select.score(X_test, y_test)
print(f"Test Accuracy (SelectKBest only): {select_test_acc:.4f}")
print()
# ========================================
# Strategy 3: FeatureUnion (PCA + SelectKBest)
# ========================================
print("=" * 80)
print("Strategy 3: FeatureUnion (PCA + SelectKBest)")
print("=" * 80)
# Create FeatureUnion with two parallel branches
feature_union = FeatureUnion([
    ('pca_branch', Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=50))
    ])),
    ('select_branch', Pipeline([
        ('scaler', StandardScaler()),
        ('select', SelectKBest(f_classif, k=30))
    ]))
])
# Full pipeline with FeatureUnion
pipeline_union = Pipeline([
    ('feature_union', feature_union),
    ('classifier', LogisticRegression(max_iter=1000))
])
# Train
pipeline_union.fit(X_train, y_train)
# Inspect FeatureUnion output shape
X_train_union = pipeline_union.named_steps['feature_union'].transform(X_train)
print(f"FeatureUnion output shape: {X_train_union.shape}")
print(f"  (50 PCA components + 30 selected features = {X_train_union.shape[1]} total)")
print()
# Evaluate
union_test_acc = pipeline_union.score(X_test, y_test)
print(f"Test Accuracy (FeatureUnion): {union_test_acc:.4f}")
print()
# ========================================
# Compare All Strategies
# ========================================
print("=" * 80)
print("Performance Comparison")
print("=" * 80)
results = pd.DataFrame({
    'Strategy': ['PCA only', 'SelectKBest only', 'FeatureUnion (PCA + SelectKBest)'],
    'Test Accuracy': [pca_test_acc, select_test_acc, union_test_acc],
    'Num Features': [50, 30, 80]
})
print(results)
print()
# Improvement
best_single = max(pca_test_acc, select_test_acc)
improvement = union_test_acc - best_single
print(f"Best single strategy: {best_single:.4f}")
print(f"FeatureUnion:         {union_test_acc:.4f}")
print(f"Improvement:          {improvement:.4f} ({100*improvement/best_single:.1f}%)")
print()


### 📝 Implementation Part 3

**Purpose:** Continue implementation

**Key implementation details below.**

In [ ]:
# ========================================
# Visualization 1: Performance Comparison
# ========================================
plt.figure(figsize=(10, 6))
bars = plt.bar(results['Strategy'], results['Test Accuracy'], 
               color=['#3498db', '#2ecc71', '#e74c3c'], edgecolor='black', linewidth=1.5)
plt.ylabel('Test Accuracy', fontsize=11, weight='bold')
plt.xlabel('Strategy', fontsize=11, weight='bold')
plt.title('Feature Extraction Strategy Comparison', fontsize=12, weight='bold')
plt.ylim([0.5, 1.0])
plt.xticks(rotation=15, ha='right')
plt.grid(alpha=0.3, axis='y')
# Annotate bars
for bar, acc in zip(bars, results['Test Accuracy']):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{acc:.3f}', ha='center', va='bottom', fontsize=10, weight='bold')
plt.tight_layout()
plt.show()
print("✅ Visualization 1: Performance comparison")
print()
# ========================================
# GridSearchCV: Tune FeatureUnion Hyperparameters
# ========================================
print("=" * 80)
print("GridSearchCV: Tuning FeatureUnion Hyperparameters")
print("=" * 80)
# Define parameter grid
param_grid = {
    'feature_union__pca_branch__pca__n_components': [30, 50, 70],
    'feature_union__select_branch__select__k': [20, 30, 40],
    'classifier__C': [0.1, 1.0, 10.0]
}
grid_search = GridSearchCV(
    pipeline_union,
    param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)
print("Parameter grid:")
for param, values in param_grid.items():
    print(f"  {param}: {values}")
print()
print("Running GridSearchCV (5-fold CV, 3×3×3 = 27 combinations)...")
grid_search.fit(X_train, y_train)
print("✓ GridSearchCV complete!")
print()
# Best parameters
print("Best parameters:")
for param, value in grid_search.best_params_.items():
    print(f"  {param}: {value}")
print()
print(f"Best cross-validation score: {grid_search.best_score_:.4f}")
print(f"Test score with best params:  {grid_search.score(X_test, y_test):.4f}")
print()


### 📝 Implementation Part 4

**Purpose:** Continue implementation

**Key implementation details below.**

In [ ]:
# ========================================
# Visualization 2: GridSearchCV Results
# ========================================
# Extract results
cv_results = pd.DataFrame(grid_search.cv_results_)
cv_results_subset = cv_results[['param_feature_union__pca_branch__pca__n_components',
                                 'param_feature_union__select_branch__select__k',
                                 'param_classifier__C',
                                 'mean_test_score']].copy()
cv_results_subset.columns = ['PCA_components', 'SelectK', 'LogReg_C', 'CV_Score']
# Pivot for heatmap (fix C=1.0 for visualization)
heatmap_data = cv_results_subset[cv_results_subset['LogReg_C'] == 1.0].pivot(
    index='PCA_components', columns='SelectK', values='CV_Score'
)
plt.figure(figsize=(8, 6))
sns.heatmap(heatmap_data, annot=True, fmt='.3f', cmap='YlGnBu', cbar_kws={'label': 'CV Accuracy'})
plt.title('GridSearchCV: PCA Components vs SelectKBest k (C=1.0)', fontsize=12, weight='bold')
plt.xlabel('SelectKBest k', fontsize=11, weight='bold')
plt.ylabel('PCA n_components', fontsize=11, weight='bold')
plt.tight_layout()
plt.show()
print("✅ Visualization 2: GridSearchCV heatmap")
print()
# ========================================
# Advanced: FeatureUnion with Custom Transformer
# ========================================
print("=" * 80)
print("Advanced: FeatureUnion with Custom Transformer")
print("=" * 80)
# Generate data with wafer_id for custom transformer
n_wafers = 10
wafer_ids = np.repeat(range(n_wafers), n_samples // n_wafers)
data_with_wafers = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(n_features)])
data_with_wafers['wafer_id'] = wafer_ids
# Rebuild train/test
X_train_df = data_with_wafers[:train_size]
X_test_df = data_with_wafers[train_size:]
# Custom transformer for wafer statistics
class WaferStatistics(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_out = X.copy()
        # Wafer-level mean of first 10 features
        wafer_means = X_out.groupby('wafer_id')[[f'feature_{i}' for i in range(10)]].transform('mean')
        wafer_means.columns = [f'{col}_wafer_mean' for col in wafer_means.columns]
        X_out = pd.concat([X_out, wafer_means], axis=1)
        return X_out.drop('wafer_id', axis=1).values
# FeatureUnion with custom transformer
advanced_union = FeatureUnion([
    ('statistical', Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=40))
    ])),
    ('wafer_features', WaferStatistics())
])
# Full pipeline
advanced_pipeline = Pipeline([
    ('feature_union', advanced_union),
    ('classifier', LogisticRegression(max_iter=1000))
])
# Train
advanced_pipeline.fit(X_train_df, y_train)
advanced_test_acc = advanced_pipeline.score(X_test_df, y_test)
print(f"Test Accuracy (FeatureUnion + Custom): {advanced_test_acc:.4f}")
print(f"Baseline (PCA only):                   {pca_test_acc:.4f}")
print(f"Improvement:                           {advanced_test_acc - pca_test_acc:.4f}")
print()


### 📝 Implementation Part 5

**Purpose:** Continue implementation

**Key implementation details below.**

In [ ]:
# ========================================
# Key Takeaways
# ========================================
print("=" * 80)
print("Key Takeaways: FeatureUnion")
print("=" * 80)
print("1. ✅ FeatureUnion: Parallel feature extraction → Concatenate outputs")
print("2. ✅ Captures diverse representations: PCA (variance) + SelectKBest (target correlation)")
print("3. ✅ Typical improvement: 3-10% accuracy over single strategy")
print("4. ✅ GridSearchCV compatible: Tune all hyperparameters jointly")
print("5. ✅ Custom transformers integrate seamlessly (wafer statistics, domain features)")
print("6. 🏭 Semiconductor: Spatial PCA + Parametric selection → 5-15% yield prediction boost")
print("7. ⚙️ Computation: Parallel branches → No performance overhead")
print("=" * 80)


## 🏭 Production Deployment: Serialization, Versioning, Monitoring

### **The Production Challenge**

**Development (easy):**
```python
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.3f}")
```

**Production (hard):**
- Save trained pipeline → Load on different server → Predict on new data
- Handle version changes (pipeline v1 → v2)
- Monitor performance drift (training accuracy 0.95 → production 0.70)
- Rollback if new version degrades performance
- Audit trail: Which pipeline version made this prediction?

---

### **Pipeline Serialization**

**Method 1: Joblib (Recommended)**

```python
import joblib

# Save pipeline
joblib.dump(pipeline, 'model_v1_20250109.pkl')

# Load pipeline (on production server)
loaded_pipeline = joblib.load('model_v1_20250109.pkl')

# Predict
y_pred = loaded_pipeline.predict(X_new)
```

**Advantages:**
- ✅ Fast serialization for large numpy arrays
- ✅ Efficient compression
- ✅ Standard in sklearn ecosystem

**Disadvantages:**
- ❌ Not cross-version compatible (joblib 1.0 vs 1.3)
- ❌ Python version dependent (trained on 3.8, load on 3.11 may fail)

---

**Method 2: Pickle (Built-in)**

```python
import pickle

# Save
with open('model_v1.pkl', 'wb') as f:
    pickle.dump(pipeline, f)

# Load
with open('model_v1.pkl', 'rb') as f:
    pipeline = pickle.load(f)
```

**Advantages:**
- ✅ Built-in (no dependencies)
- ✅ Works with any Python object

**Disadvantages:**
- ❌ Slower than joblib for large arrays
- ❌ Less compression
- ❌ Security risk (untrusted pickles can execute arbitrary code)

---

**Method 3: ONNX (Cross-Platform)**

```python
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# Convert to ONNX
initial_type = [('float_input', FloatTensorType([None, n_features]))]
onnx_model = convert_sklearn(pipeline, initial_types=initial_type)

# Save
with open('model_v1.onnx', 'wb') as f:
    f.write(onnx_model.SerializeToString())

# Load and predict (works in C++, Java, JavaScript!)
import onnxruntime as rt
sess = rt.InferenceSession('model_v1.onnx')
input_name = sess.get_inputs()[0].name
y_pred = sess.run(None, {input_name: X_new.astype(np.float32)})[0]
```

**Advantages:**
- ✅ Cross-platform (Python, C++, Java, JavaScript, mobile)
- ✅ Optimized inference (faster than Python)
- ✅ Language-agnostic deployment

**Disadvantages:**
- ❌ Not all sklearn transformers supported
- ❌ Custom transformers require manual conversion
- ❌ More complex setup

---

### **Versioning Strategy**

**File naming convention:**

```
model_<version>_<date>_<git_commit>.pkl

Examples:
- model_v1.0.0_20250109_a3f8c21.pkl
- model_v1.1.0_20250115_b9d4e12.pkl
- model_v2.0.0_20250201_c7a5f33.pkl
```

**Metadata file (JSON):**

```json
{
  "model_version": "v1.0.0",
  "training_date": "2025-01-09",
  "git_commit": "a3f8c21",
  "training_data": {
    "n_samples": 10000,
    "n_features": 80,
    "class_distribution": {"0": 4000, "1": 6000}
  },
  "hyperparameters": {
    "pca__n_components": 50,
    "select__k": 30,
    "classifier__C": 1.0
  },
  "performance": {
    "train_accuracy": 0.952,
    "test_accuracy": 0.938,
    "cv_mean": 0.945,
    "cv_std": 0.012
  },
  "feature_names": ["Vdd_min", "Vdd_max", "Idd_active", ...],
  "dependencies": {
    "sklearn": "1.3.0",
    "numpy": "1.24.0",
    "python": "3.8.16"
  }
}
```

**Benefit:** Complete audit trail for each model version.

---

### **Caching Expensive Transformations**

**Problem:** TF-IDF on 1M documents takes 10 minutes

**Solution:** Cache with `memory` parameter

```python
from sklearn.pipeline import Pipeline
from joblib import Memory

# Create cache directory
memory = Memory(location='/tmp/pipeline_cache', verbose=0)

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=10000)),  # Expensive
    ('classifier', LogisticRegression())
], memory=memory)

# First fit: Compute TF-IDF (10 minutes)
pipeline.fit(X_train, y_train)

# GridSearchCV with 5-fold CV:
# - Without caching: 5 × 10 minutes = 50 minutes
# - With caching: 1 × 10 minutes (first fold) + 4 × 0 minutes (cached) = 10 minutes
param_grid = {'classifier__C': [0.1, 1.0, 10.0]}
grid_search = GridSearchCV(pipeline, param_grid, cv=5)
grid_search.fit(X_train, y_train)  # Fast! (TF-IDF cached)
```

**Speedup:** 5x for 5-fold CV, 10x for 10-fold CV.

---

### **Production Monitoring**

**1. Input Drift Detection**

**Problem:** Training data distribution ≠ production data distribution

**Solution:** Monitor feature statistics

```python
# Training statistics
train_stats = {
    'mean': X_train.mean(axis=0),
    'std': X_train.std(axis=0),
    'min': X_train.min(axis=0),
    'max': X_train.max(axis=0)
}

# Production monitoring
X_prod_batch = get_production_data()  # Last 1000 predictions
prod_stats = {
    'mean': X_prod_batch.mean(axis=0),
    'std': X_prod_batch.std(axis=0)
}

# Drift detection
drift = np.abs(train_stats['mean'] - prod_stats['mean']) / train_stats['std']
if np.any(drift > 3):  # 3-sigma threshold
    alert("Input drift detected! Retrain model.")
```

---

**2. Prediction Drift Detection**

**Problem:** Model performance degrades over time

**Solution:** Monitor prediction distribution

```python
# Training prediction distribution
train_pred = pipeline.predict_proba(X_train)[:, 1]
train_pred_mean = train_pred.mean()

# Production monitoring
prod_pred = pipeline.predict_proba(X_prod_batch)[:, 1]
prod_pred_mean = prod_pred.mean()

# Check drift
if abs(prod_pred_mean - train_pred_mean) > 0.1:
    alert("Prediction drift detected! Retrain model.")
```

---

**3. Performance Monitoring (with ground truth)**

**Scenario:** Semiconductor testing (immediate feedback)

```python
# Predict on test floor
y_pred = pipeline.predict(X_prod)

# After testing complete, get actual yield
y_actual = get_actual_yield()  # 24 hours later

# Monitor accuracy
prod_accuracy = accuracy_score(y_actual, y_pred)

if prod_accuracy < 0.85:  # Threshold
    alert("Model accuracy dropped! Retrain urgently.")
```

---

### **A/B Testing Pipelines**

**Scenario:** Testing new pipeline version (v2) against production (v1)

```python
import random

def predict_with_ab_test(X_new, pipeline_v1, pipeline_v2):
    """Route 10% traffic to v2, 90% to v1."""
    if random.random() < 0.1:  # 10% to v2
        y_pred = pipeline_v2.predict(X_new)
        version = 'v2'
    else:  # 90% to v1
        y_pred = pipeline_v1.predict(X_new)
        version = 'v1'
    
    # Log for analysis
    log_prediction(X_new, y_pred, version)
    
    return y_pred

# After 1 week, compare performance
v1_accuracy = compute_accuracy('v1')
v2_accuracy = compute_accuracy('v2')

if v2_accuracy > v1_accuracy + 0.02:  # 2% improvement
    promote_to_production('v2')
else:
    keep_current_version('v1')
```

---

### **Rollback Strategy**

**Problem:** New model version (v2) performs worse in production

**Solution:** Keep previous versions, enable instant rollback

```bash
# Model storage
models/
  ├── model_v1.0.0_20250109.pkl  (stable)
  ├── model_v1.1.0_20250115.pkl  (stable)
  ├── model_v2.0.0_20250201.pkl  (current, degraded)
  └── current_model.pkl -> model_v2.0.0_20250201.pkl  (symlink)

# Rollback (instant!)
$ ln -sf model_v1.1.0_20250115.pkl current_model.pkl

# Application always loads current_model.pkl (no code change needed)
pipeline = joblib.load('models/current_model.pkl')
```

**Benefit:** Rollback in seconds, zero downtime.

---

### **Docker Containerization**

**Dockerfile for pipeline deployment:**

```dockerfile
FROM python:3.8-slim

# Install dependencies
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Copy model and code
COPY models/model_v1.0.0.pkl /app/model.pkl
COPY app.py /app/

# Expose API
WORKDIR /app
EXPOSE 8000

CMD ["uvicorn", "app:app", "--host", "0.0.0.0", "--port", "8000"]
```

**API endpoint (FastAPI):**

```python
from fastapi import FastAPI
import joblib
import numpy as np

app = FastAPI()
pipeline = joblib.load('model.pkl')

@app.post("/predict")
def predict(features: list[float]):
    X = np.array(features).reshape(1, -1)
    y_pred = pipeline.predict(X)
    y_prob = pipeline.predict_proba(X)
    return {
        "prediction": int(y_pred[0]),
        "probability": float(y_prob[0, 1])
    }
```

**Deploy:**

```bash
docker build -t yield-predictor:v1 .
docker run -p 8000:8000 yield-predictor:v1

# Test
curl -X POST http://localhost:8000/predict \
  -H "Content-Type: application/json" \
  -d '{"features": [1.0, 1.2, 50, 1.0, 2000, 85]}'
```

---

### **CI/CD for ML Pipelines**

**GitHub Actions workflow:**

```yaml
name: ML Pipeline CI/CD

on:
  push:
    branches: [main]

jobs:
  test-and-deploy:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
      
      - name: Set up Python
        uses: actions/setup-python@v4
        with:
          python-version: '3.8'
      
      - name: Install dependencies
        run: pip install -r requirements.txt
      
      - name: Run unit tests
        run: pytest tests/
      
      - name: Train pipeline
        run: python train_pipeline.py
      
      - name: Validate performance
        run: |
          python validate.py
          if [ $? -ne 0 ]; then
            echo "Performance below threshold!"
            exit 1
          fi
      
      - name: Build Docker image
        run: docker build -t yield-predictor:${{ github.sha }} .
      
      - name: Push to registry
        run: |
          docker tag yield-predictor:${{ github.sha }} myregistry/yield-predictor:latest
          docker push myregistry/yield-predictor:latest
      
      - name: Deploy to production
        run: kubectl set image deployment/yield-predictor app=myregistry/yield-predictor:latest
```

---

### **Semiconductor Production Example**

**Real-world deployment flow:**

```mermaid
graph LR
    A[Wafer Test Data<br/>STDF Files] --> B[Pipeline v1.2<br/>Preprocessing + Model]
    B --> C{Yield Prediction<br/>Pass/Fail}
    C -->|Pass| D[Bin 1<br/>Ship to Customer]
    C -->|Fail| E[Bin 2<br/>Scrap]
    
    B --> F[Monitoring<br/>Accuracy, Drift, Latency]
    F --> G{Performance OK?}
    G -->|Yes| H[Continue]
    G -->|No| I[Alert Engineers<br/>Retrain or Rollback]
    
    style A fill:#ffe6e6
    style B fill:#e6f3ff
    style C fill:#fff3e6
    style D fill:#90EE90
    style E fill:#FFB6C1
    style F fill:#e6e6fa
    style I fill:#ff6b6b
```

**Production requirements:**
- **Latency:** <50ms per device (1M devices/day = 11.5 devices/sec)
- **Availability:** 99.9% uptime ($1M loss per hour downtime)
- **Accuracy:** >95% (5% error = $10M-$50M annual yield loss)
- **Audit trail:** Every prediction logged (regulatory compliance)

---

### **Next: Complete Production Example**

We'll build:
1. **Full pipeline** with preprocessing + model
2. **Serialization** with versioning and metadata
3. **Load and predict** simulation
4. **Performance monitoring** with drift detection

Let's implement! 🚀

### 📝 What's Happening in This Code?

**Purpose:** Demonstrate **complete production pipeline deployment** with serialization, versioning, metadata tracking, and monitoring.

**Key Points:**
- **End-to-end pipeline:** Custom transformer → ColumnTransformer → FeatureUnion → Model
- **Serialization:** Save with joblib, include version metadata (training date, git commit, performance metrics)
- **Production simulation:** Load saved pipeline, predict on new data, measure latency
- **Monitoring:** Track input drift (feature statistics), prediction drift (distribution changes)
- **Rollback capability:** Save multiple versions, enable instant fallback if new version degrades

**Why This Matters:** Production ML is 10% training, 90% deployment/monitoring. Pipelines must be versionable, auditable, and monitorable. For semiconductor manufacturing, production systems predict 1M+ devices/day with <50ms latency and 99.9% uptime ($1M/hour downtime cost). Proper versioning prevents $10M-$50M yield losses from model degradation.

### 📝 Implementation

**Purpose:** Core implementation with detailed code

**Key implementation details below.**

In [ ]:
import joblib
import json
import time
from datetime import datetime
# ========================================
# Build Complete Production Pipeline
# ========================================
print("=" * 80)
print("Building Complete Production Pipeline")
print("=" * 80)
# Generate production-grade semiconductor data
np.random.seed(42)
n_samples = 2000
n_wafers = 20
# Parametric data
wafer_ids = np.repeat(range(n_wafers), n_samples // n_wafers)
Vdd_min = 1.0 + np.random.randn(n_samples) * 0.1
Vdd_max = 1.2 + np.random.randn(n_samples) * 0.1
Idd_active = 50 + np.random.randn(n_samples) * 10
Idd_standby = 1 + np.random.randn(n_samples) * 0.3
freq_max = 2000 + np.random.randn(n_samples) * 200
temp = 85 + np.random.randn(n_samples) * 5
# Categorical data
site_ids = np.random.choice(['Site_A', 'Site_B', 'Site_C'], n_samples)
product_types = np.random.choice(['Type_X', 'Type_Y'], n_samples)
data = pd.DataFrame({
    'wafer_id': wafer_ids,
    'Vdd_min': Vdd_min,
    'Vdd_max': Vdd_max,
    'Idd_active': Idd_active,
    'Idd_standby': Idd_standby,
    'freq_max': freq_max,
    'temp': temp,
    'site_id': site_ids,
    'product_type': product_types
})
# Generate yield labels
site_effect = (data['site_id'] == 'Site_A').astype(float) * 0.3
product_effect = (data['product_type'] == 'Type_Y').astype(float) * 0.2
y_prob = 1 / (1 + np.exp(-(
    5 * (data['Vdd_min'] - 1.0) +
    0.05 * (data['Idd_active'] - 50) +
    0.002 * (data['freq_max'] - 2000) +
    site_effect + product_effect - 0.5
)))
y = (y_prob > 0.5).astype(int)
print(f"Dataset: {n_samples} devices from {n_wafers} wafers")
print(f"Features: {list(data.columns)}")
print(f"Target distribution: {np.sum(y==0)} fails, {np.sum(y==1)} passes")
print()
# Train/test split
train_size = int(0.8 * n_samples)
X_train, X_test = data[:train_size], data[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
# ========================================
# Define Complete Pipeline
# ========================================
print("=" * 80)
print("Pipeline Architecture")
print("=" * 80)
# Define column groups
numeric_features = ['Vdd_min', 'Vdd_max', 'Idd_active', 'Idd_standby', 'freq_max', 'temp']
categorical_features = ['site_id', 'product_type']
wafer_features = ['wafer_id', 'Vdd_min', 'Vdd_max', 'Idd_active', 'Idd_standby', 'freq_max', 'temp']
# Helper to extract numeric columns after feature engineering


### 📝 Function: select_numeric_after_engineering

**Purpose:** Continue implementation

**Key implementation details below.**

In [ ]:
def select_numeric_after_engineering(X):
    # After SemiconductorFeatureEngineer, we have original + engineered features
    numeric_cols = numeric_features + ['Vdd_min_detrended', 'Vdd_ratio', 'Idd_efficiency', 
                                        'power_per_freq', 'power_estimate', 'freq_power_stress', 'temp_leakage']
    return X[numeric_cols].values
# Build pipeline
production_pipeline = Pipeline([
    # Step 1: Domain-specific feature engineering
    ('feature_engineer', SemiconductorFeatureEngineer(
        spatial_detrending=True,
        create_ratios=True,
        create_interactions=True
    )),
    
    # Step 2: Extract numeric features
    ('select_numeric', FunctionTransformer(select_numeric_after_engineering)),
    
    # Step 3: FeatureUnion (PCA + SelectKBest)
    ('feature_union', FeatureUnion([
        ('pca_branch', Pipeline([
            ('scaler', StandardScaler()),
            ('pca', PCA(n_components=10))
        ])),
        ('select_branch', Pipeline([
            ('scaler', StandardScaler()),
            ('select', SelectKBest(f_classif, k=8))
        ]))
    ])),
    
    # Step 4: Final classifier
    ('classifier', RandomForestClassifier(n_estimators=100, max_depth=15, random_state=42))
])
print("Pipeline steps:")
for i, (name, step) in enumerate(production_pipeline.steps, 1):
    print(f"  {i}. {name}: {type(step).__name__}")
print()
# ========================================
# Train Pipeline
# ========================================
print("=" * 80)
print("Training Production Pipeline")
print("=" * 80)
start_time = time.time()
production_pipeline.fit(X_train, y_train)
train_time = time.time() - start_time
print(f"✓ Training complete in {train_time:.2f} seconds")
print()
# Evaluate
train_acc = production_pipeline.score(X_train, y_train)
test_acc = production_pipeline.score(X_test, y_test)
print(f"Train Accuracy: {train_acc:.4f}")
print(f"Test Accuracy:  {test_acc:.4f}")
print()
# ========================================
# Serialize Pipeline with Metadata
# ========================================
print("=" * 80)
print("Serializing Pipeline with Metadata")
print("=" * 80)
# Model version info
model_version = "v1.0.0"
training_date = datetime.now().strftime("%Y%m%d")
git_commit = "a3f8c21"  # Simulated
# File names
model_filename = f"model_{model_version}_{training_date}_{git_commit}.pkl"
metadata_filename = f"model_{model_version}_{training_date}_{git_commit}_metadata.json"
# Save pipeline
joblib.dump(production_pipeline, model_filename)
print(f"✓ Pipeline saved: {model_filename}")
# Compute additional metrics
from sklearn.metrics import precision_score, recall_score, f1_score
y_pred_train = production_pipeline.predict(X_train)
y_pred_test = production_pipeline.predict(X_test)
train_precision = precision_score(y_train, y_pred_train)
train_recall = recall_score(y_train, y_pred_train)
train_f1 = f1_score(y_train, y_pred_train)
test_precision = precision_score(y_test, y_pred_test)
test_recall = recall_score(y_test, y_pred_test)
test_f1 = f1_score(y_test, y_pred_test)
# Create metadata
metadata = {
    "model_version": model_version,
    "training_date": training_date,
    "git_commit": git_commit,
    "training_time_seconds": round(train_time, 2),
    "training_data": {
        "n_samples": len(X_train),
        "n_features_raw": len(data.columns),
        "n_features_engineered": 13,
        "n_features_final": 18,
        "class_distribution": {
            "fail": int(np.sum(y_train == 0)),
            "pass": int(np.sum(y_train == 1))
        }
    },
    "hyperparameters": {
        "feature_engineer__spatial_detrending": True,
        "feature_union__pca_branch__pca__n_components": 10,
        "feature_union__select_branch__select__k": 8,
        "classifier__n_estimators": 100,
        "classifier__max_depth": 15
    },
    "performance": {
        "train": {
            "accuracy": round(train_acc, 4),
            "precision": round(train_precision, 4),
            "recall": round(train_recall, 4),
            "f1": round(train_f1, 4)
        },
        "test": {
            "accuracy": round(test_acc, 4),
            "precision": round(test_precision, 4),
            "recall": round(test_recall, 4),
            "f1": round(test_f1, 4)
        }
    },
    "feature_names": list(data.columns),
    "dependencies": {
        "sklearn": "1.3.0",
        "numpy": "1.24.0",
        "pandas": "2.0.0",
        "python": "3.8.16"
    }
}
# Save metadata
with open(metadata_filename, 'w') as f:
    json.dump(metadata, f, indent=2)
print(f"✓ Metadata saved: {metadata_filename}")
print()
# Display metadata sample
print("Metadata sample:")
print(json.dumps(metadata["performance"], indent=2))
print()


### 📝 Implementation Part 3

**Purpose:** Continue implementation

**Key implementation details below.**

In [ ]:
# ========================================
# Production Simulation: Load and Predict
# ========================================
print("=" * 80)
print("Production Simulation: Load and Predict")
print("=" * 80)
# Simulate loading on production server
loaded_pipeline = joblib.load(model_filename)
print(f"✓ Pipeline loaded from {model_filename}")
# Load metadata
with open(metadata_filename, 'r') as f:
    loaded_metadata = json.load(f)
print(f"✓ Metadata loaded: version {loaded_metadata['model_version']}, trained {loaded_metadata['training_date']}")
print()
# Simulate new production data (10 devices)
X_production = X_test.iloc[:10].copy()
# Measure inference latency
start_time = time.time()
y_pred_prod = loaded_pipeline.predict(X_production)
y_prob_prod = loaded_pipeline.predict_proba(X_production)
inference_time = (time.time() - start_time) * 1000  # ms
print(f"Production inference: {len(X_production)} devices")
print(f"Latency: {inference_time:.2f} ms total ({inference_time/len(X_production):.2f} ms/device)")
print()
print("Sample predictions:")
results_df = pd.DataFrame({
    'Device_ID': range(1, 11),
    'Prediction': y_pred_prod,
    'Pass_Probability': y_prob_prod[:, 1]
})
print(results_df)
print()
# ========================================
# Monitoring: Input Drift Detection
# ========================================
print("=" * 80)
print("Monitoring: Input Drift Detection")
print("=" * 80)
# Compute training statistics
train_stats = {
    'mean': X_train[numeric_features].mean().to_dict(),
    'std': X_train[numeric_features].std().to_dict(),
    'min': X_train[numeric_features].min().to_dict(),
    'max': X_train[numeric_features].max().to_dict()
}
# Simulate production batch (last 100 test samples)
X_prod_batch = X_test.iloc[-100:]
prod_stats = {
    'mean': X_prod_batch[numeric_features].mean().to_dict(),
    'std': X_prod_batch[numeric_features].std().to_dict()
}
# Drift detection (3-sigma rule)
print("Input drift analysis:")
drift_detected = False
for feature in numeric_features:
    train_mean = train_stats['mean'][feature]
    train_std = train_stats['std'][feature]
    prod_mean = prod_stats['mean'][feature]
    
    drift = abs(prod_mean - train_mean) / train_std
    status = "⚠️ DRIFT" if drift > 3 else "✓ OK"
    
    print(f"  {feature:15s}: drift = {drift:.2f}σ  {status}")
    
    if drift > 3:
        drift_detected = True
print()
if drift_detected:
    print("⚠️ WARNING: Input drift detected! Consider retraining model.")
else:
    print("✓ No significant input drift detected.")
print()


### 📝 Implementation Part 4

**Purpose:** Continue implementation

**Key implementation details below.**

In [ ]:
# ========================================
# Monitoring: Prediction Drift
# ========================================
print("=" * 80)
print("Monitoring: Prediction Drift")
print("=" * 80)
# Training prediction distribution
y_prob_train = loaded_pipeline.predict_proba(X_train)[:, 1]
train_pred_mean = y_prob_train.mean()
train_pred_std = y_prob_train.std()
# Production prediction distribution
y_prob_prod_batch = loaded_pipeline.predict_proba(X_prod_batch)[:, 1]
prod_pred_mean = y_prob_prod_batch.mean()
prod_pred_std = y_prob_prod_batch.std()
print(f"Training predictions:   mean = {train_pred_mean:.3f}, std = {train_pred_std:.3f}")
print(f"Production predictions: mean = {prod_pred_mean:.3f}, std = {prod_pred_std:.3f}")
print()
# Drift check
pred_drift = abs(prod_pred_mean - train_pred_mean)
if pred_drift > 0.1:  # 10% threshold
    print(f"⚠️ WARNING: Prediction drift = {pred_drift:.3f} (threshold: 0.1)")
    print("   Consider retraining or rolling back to previous model version.")
else:
    print(f"✓ Prediction drift = {pred_drift:.3f} (within acceptable range)")
print()
# ========================================
# Visualization: Prediction Distribution
# ========================================
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
# Training distribution
axes[0].hist(y_prob_train, bins=30, alpha=0.7, color='steelblue', edgecolor='black')
axes[0].axvline(train_pred_mean, color='red', linestyle='--', linewidth=2, label=f'Mean: {train_pred_mean:.3f}')
axes[0].set_xlabel('Predicted Probability (Pass)', fontsize=10, weight='bold')
axes[0].set_ylabel('Frequency', fontsize=10, weight='bold')
axes[0].set_title('Training Predictions Distribution', fontsize=11, weight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)
# Production distribution
axes[1].hist(y_prob_prod_batch, bins=30, alpha=0.7, color='seagreen', edgecolor='black')
axes[1].axvline(prod_pred_mean, color='red', linestyle='--', linewidth=2, label=f'Mean: {prod_pred_mean:.3f}')
axes[1].set_xlabel('Predicted Probability (Pass)', fontsize=10, weight='bold')
axes[1].set_ylabel('Frequency', fontsize=10, weight='bold')
axes[1].set_title('Production Predictions Distribution', fontsize=11, weight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)
plt.tight_layout()
plt.show()
print("✅ Visualization: Prediction distributions (train vs production)")
print()
# ========================================
# Simulate Rollback Scenario
# ========================================
print("=" * 80)
print("Simulate Rollback Scenario")
print("=" * 80)
# Simulate v2 with worse performance
model_v2_acc = test_acc - 0.05  # 5% degradation
print(f"Production scenario:")
print(f"  v1.0.0 test accuracy: {test_acc:.4f}")
print(f"  v2.0.0 test accuracy: {model_v2_acc:.4f} (deployed to production)")
print()
print("⚠️ Alert: v2.0.0 performance degraded by 5%!")
print("Action: Rolling back to v1.0.0...")
print()
# Simulate rollback (just re-load v1.0.0)
rollback_pipeline = joblib.load(model_filename)
rollback_acc = rollback_pipeline.score(X_test, y_test)
print(f"✓ Rollback complete: v1.0.0 accuracy = {rollback_acc:.4f}")
print("✓ Production restored to stable version")
print()


### 📝 Implementation Part 5

**Purpose:** Continue implementation

**Key implementation details below.**

In [ ]:
# ========================================
# Key Takeaways
# ========================================
print("=" * 80)
print("Key Takeaways: Production Deployment")
print("=" * 80)
print("1. ✅ Serialization: joblib for sklearn pipelines (fast, efficient)")
print("2. ✅ Versioning: model_version_date_commit.pkl + metadata.json")
print("3. ✅ Metadata: Training date, hyperparameters, performance, dependencies")
print("4. ✅ Monitoring: Input drift (3σ), prediction drift (10% threshold)")
print("5. ✅ Rollback: Keep previous versions, instant fallback (symlink)")
print("6. ✅ Latency: ~2-5ms per device (meets <50ms requirement)")
print("7. 🏭 Semiconductor: 1M devices/day, 99.9% uptime, $1M/hour downtime cost")
print("8. 📊 Complete audit trail: Every model version documented")
print("=" * 80)


## 🎯 Real-World Projects: ML Pipelines & Automation

Apply ML pipeline concepts to solve real-world problems in post-silicon validation and general AI/ML domains.

---

### **Post-Silicon Validation Projects (4)**

---

#### **Project 1: Automated Multi-Stage Test Flow Pipeline**

**Objective:** Build end-to-end pipeline for wafer test → final test → binning workflow

**Business Value:** $20M-$80M annual savings from test time reduction + yield optimization

**Key Components:**
- **Wafer test preprocessing:** Spatial detrending, outlier capping, feature scaling
- **Final test preprocessing:** Temporal filtering (equipment drift), parametric ratios
- **FeatureUnion:** Combine wafer features (PCA) + final test features (SelectKBest)
- **Multi-class classifier:** Predict bin category (Bin1/Premium, Bin2/Standard, Bin3/Scrap)
- **Pipeline optimization:** Cache expensive transforms, GridSearchCV for hyperparameters

**Success Metrics:**
- Test time reduction: 20-30% (from 150s → 105s per device)
- Binning accuracy: >98% (vs 95% manual rules)
- Inference latency: <50ms per device (production requirement)

**Data Requirements:**
- STDF files: Wafer test (200 parametric tests) + Final test (150 parametric tests)
- Spatial coordinates: (wafer_id, die_x, die_y)
- Temporal data: Test timestamps, equipment tool IDs
- Target: Bin category (1-3) based on customer specifications

**Implementation Steps:**
1. Load and parse STDF files (use `pystdf` library)
2. Build custom transformers: `WaferSpatialDetrending`, `EquipmentDriftCorrection`
3. Create ColumnTransformer for numeric vs categorical features
4. Build FeatureUnion for multi-stage data fusion
5. Serialize pipeline with version metadata
6. Deploy to production test floor (Docker + FastAPI)
7. Monitor: Input drift, prediction drift, accuracy

**Advanced Features:**
- Real-time predictions on test floor (<50ms latency)
- A/B testing: Compare pipeline v1 vs v2 (10% traffic split)
- Automated retraining: Weekly schedule, trigger on accuracy drop
- Explainability: SHAP values for each bin prediction (regulatory compliance)

---

#### **Project 2: Multi-Site Yield Harmonization Pipeline**

**Objective:** Unified yield prediction model across 3 manufacturing sites with different equipment

**Business Value:** $30M-$100M from consistent yield across sites, enable capacity balancing

**Key Components:**
- **Site-specific preprocessing:** Independent StandardScaler per site (different equipment calibration)
- **FeatureUnion:** Site-agnostic features (PCA) + site-specific features (categorical encoding)
- **Domain adaptation:** Learn site-invariant representations
- **Custom transformer:** `MultiSiteNormalizer` (align distributions across sites)
- **Ensemble:** Random Forest per site + meta-learner

**Success Metrics:**
- Cross-site accuracy: >90% (model trained on Site A, tested on Site B)
- Site bias reduction: <5% accuracy drop when transferring sites
- Capacity balancing: Enable 20% yield improvement via site allocation

**Data Requirements:**
- Multi-site STDF data: Site A (US fab), Site B (Taiwan fab), Site C (Korea fab)
- Equipment metadata: Tool IDs, calibration dates, maintenance logs
- Process parameters: Temperature, pressure, chemicals (site-specific)

**Implementation Steps:**
1. EDA: Identify site-specific biases (Site A: 5% higher Vdd_min mean)
2. Build `MultiSiteNormalizer`: Align distributions using quantile mapping
3. FeatureUnion: PCA (global patterns) + Site one-hot encoding (local effects)
4. Train ensemble: Random Forest per site + Logistic Regression meta-learner
5. Validate cross-site: Train on Site A+B, test on Site C
6. Deploy unified pipeline to all 3 sites
7. Monitor: Per-site accuracy, cross-site transfer performance

---

#### **Project 3: Real-Time Adaptive Test Flow Optimization**

**Objective:** Dynamically adjust test sequence based on early parametric results (save 40% test time)

**Business Value:** $10M-$40M annual savings from reduced test time (150s → 90s per device)

**Key Components:**
- **Early stopping pipeline:** Predict final yield from first 50 tests (vs full 200 tests)
- **Sequential feature selection:** Identify minimum test set for 95% accuracy
- **Custom transformer:** `EarlyStoppingDecider` (confidence-based test termination)
- **Online learning:** Update model daily based on production feedback
- **A/B testing:** Gradually roll out early stopping (10% → 50% → 100% traffic)

**Success Metrics:**
- Test time reduction: 30-40% (devices classified as clear pass/fail early)
- Accuracy maintained: >95% (vs full test sequence)
- False negative rate: <0.1% (critical for quality)

**Data Requirements:**
- Sequential test data: Tests ordered by execution time (Test1 → Test200)
- Test correlation matrix: Identify redundant tests
- Historical yield: 1M devices with full test results

**Implementation Steps:**
1. Analyze test correlation: Group correlated tests (e.g., Vdd_min, Vdd_max)
2. Train sequential models: Predict yield after 25, 50, 100, 150 tests
3. Build confidence-based early stopping: If P(pass) > 99% or P(fail) > 99%, stop
4. Simulate savings: Compute test time reduction on historical data
5. Deploy gradual rollout: 10% traffic for 1 week, monitor accuracy
6. Full rollout: 100% traffic if accuracy >95% maintained

---

#### **Project 4: Pipeline-Based Parametric Outlier Detection**

**Objective:** Automated outlier detection for 200+ parametric tests with minimal false positives

**Business Value:** $5M-$20M from preventing yield excursions, early process issue detection

**Key Components:**
- **Multi-stage outlier detection:** Statistical (IQR) → Model-based (Isolation Forest) → Domain rules
- **Pipeline architecture:** Parallelize detection methods via FeatureUnion
- **Custom transformer:** `AdaptiveOutlierCapper` (cap at 95th percentile, not remove)
- **Explainability:** Identify root cause (which test, which wafer, which equipment)
- **Alerting:** Slack notifications for critical outliers (>10 devices/hour)

**Success Metrics:**
- Detection rate: >99% of true outliers (validated against expert labels)
- False positive rate: <1% (avoid unnecessary scrapping)
- Response time: <30 minutes from outlier detection to engineer notification

**Data Requirements:**
- Parametric test data: 200+ tests per device, 1M devices/month
- Historical outlier labels: Expert-annotated outliers (process excursions, equipment failures)
- Process context: Lot IDs, equipment tool IDs, timestamps

**Implementation Steps:**
1. Build outlier detection pipeline: IQR → Isolation Forest → One-Class SVM
2. FeatureUnion: Combine outlier scores from each method
3. Ensemble vote: Flag if 2+ methods agree on outlier
4. SHAP explanations: Which test parameters caused outlier classification
5. Integrate with manufacturing execution system (MES)
6. Alert engineers: Automated Slack/email notifications with root cause

---

### **General AI/ML Projects (4)**

---

#### **Project 5: Production-Ready Text Classification Pipeline**

**Objective:** Deploy spam detection with 99.9% uptime and <100ms latency

**Business Value:** $50M-$150M from preventing phishing attacks, improving email quality

**Key Components:**
- **Text preprocessing pipeline:** Lowercase → Remove punctuation → Tokenize → Lemmatize
- **FeatureUnion:** TF-IDF (word-level) + Character n-grams (catch obfuscations like "V1agra")
- **Caching:** Cache TF-IDF vectorizer (expensive for 1M emails)
- **Model:** Logistic Regression (fast inference) + Calibrated probabilities
- **Deployment:** Docker + Kubernetes + Horizontal autoscaling

**Success Metrics:**
- Accuracy: >99% (F1 score for spam detection)
- Latency: <100ms per email (production SLA)
- Uptime: 99.9% (8.76 hours downtime/year max)

**Implementation Steps:**
1. Build preprocessing pipeline: Custom tokenizer + Lemmatizer
2. FeatureUnion: TF-IDF + Character n-grams (2-4 chars)
3. GridSearchCV: Tune TF-IDF max_features, Logistic Regression C
4. Serialize pipeline + metadata (version, training date)
5. Dockerize: FastAPI endpoint, load balancer
6. Deploy Kubernetes: 10 replicas, autoscale to 50 on high traffic
7. Monitor: Latency (p50, p99), error rate, prediction drift

---

#### **Project 6: Automated Feature Engineering Pipeline for Tabular Data**

**Objective:** AutoML-style feature engineering for arbitrary tabular datasets

**Business Value:** $20M-$60M from democratizing ML (non-experts can build production models)

**Key Components:**
- **Auto feature engineering:** Detect column types (numeric, categorical, datetime, text)
- **Pipeline generator:** Automatically create ColumnTransformer based on column types
- **Feature interactions:** Automatically generate polynomial features, ratios
- **Feature selection:** Automatic RFE (Recursive Feature Elimination)
- **Model selection:** Try 5 models (Logistic Regression, Random Forest, XGBoost, LightGBM, CatBoost)

**Success Metrics:**
- Accuracy: Within 5% of expert-tuned models (on 20 benchmark datasets)
- Speed: <10 minutes for full pipeline (1M samples, 100 features)
- Usability: 3 lines of code for end-to-end training

**Implementation Steps:**
1. Build `AutoFeatureEngineer`: Detect column types, generate transformers
2. Implement `FeatureInteractionGenerator`: Polynomial, ratios, interactions
3. Integrate feature selection: RFE with CV
4. Model selection: GridSearchCV across 5 models
5. Serialize best pipeline: Save model + feature engineering
6. Package as library: `pip install auto-ml-pipeline`

---

#### **Project 7: Multi-Modal Recommendation Pipeline**

**Objective:** Combine user behavior + product features + text reviews for e-commerce recommendations

**Business Value:** $100M-$300M from 10-20% conversion rate improvement

**Key Components:**
- **FeatureUnion:** User embeddings (behavior) + Product embeddings (images) + Review embeddings (text)
- **Custom transformers:** `UserBehaviorEncoder`, `ProductImageEncoder`, `ReviewTextEncoder`
- **Late fusion:** Concatenate embeddings, train neural network
- **A/B testing:** 20% traffic to new pipeline, compare CTR vs baseline
- **Real-time inference:** <50ms latency for 1M users/day

**Success Metrics:**
- Click-through rate (CTR): +15% vs baseline (collaborative filtering)
- Conversion rate: +10% (users who click → purchase)
- Latency: <50ms per recommendation (production SLA)

**Implementation Steps:**
1. Build user behavior encoder: Embed 100+ behavior features with MLP
2. Build product image encoder: ResNet50 embeddings (2048-dim)
3. Build review text encoder: BERT embeddings (768-dim)
4. FeatureUnion: Concatenate embeddings (100 + 2048 + 768 = 2916-dim)
5. Train neural network: 3-layer MLP → predict click probability
6. Serialize pipeline: joblib for encoders, TorchScript for neural network
7. Deploy: Docker + Kubernetes, autoscale to 100 replicas
8. A/B test: 20% traffic, monitor CTR/conversion rate

---

#### **Project 8: Time Series Forecasting Pipeline with Seasonality**

**Objective:** Sales forecasting with automated seasonality detection and drift monitoring

**Business Value:** $30M-$100M from optimized inventory (reduce stockouts + excess inventory)

**Key Components:**
- **Custom transformer:** `SeasonalityDetector` (detect daily, weekly, yearly patterns)
- **Rolling window features:** `RollingStatistics` (mean, std, quantiles over last 7/30/90 days)
- **Lag features:** Previous values as predictors (lag-1, lag-7, lag-30)
- **Model:** XGBoost for regression (handles non-linear seasonality)
- **Drift monitoring:** Retrain weekly, alert if MAPE > 15%

**Success Metrics:**
- Forecast accuracy: MAPE < 10% (Mean Absolute Percentage Error)
- Inventory optimization: 20% reduction in excess inventory
- Stockout prevention: 95% fill rate (vs 85% baseline)

**Implementation Steps:**
1. Build `SeasonalityDetector`: FFT to detect periodicities
2. Build `RollingStatistics`: Compute rolling mean, std, quantiles
3. Create lag features: lag-1, lag-7, lag-30 (capture short/long-term patterns)
4. Pipeline: Seasonality + Rolling + Lag → XGBoost
5. GridSearchCV: Tune XGBoost depth, learning rate
6. Deploy: Daily batch predictions (forecast next 30 days)
7. Monitor: MAPE, drift in sales patterns (Black Friday spike detection)

---

## 🔑 Key Takeaways: ML Pipelines & Automation

### **Core Principles**

1. **✅ Pipeline = Reproducibility + Maintainability + Deployability**
   - Single object encapsulates entire workflow (preprocessing + model)
   - Serialize once, deploy anywhere (dev → staging → production)
   - Version control with metadata (training date, hyperparameters, performance)

2. **✅ Prevent Data Leakage with Sequential Fitting**
   - fit() on training data only → learn parameters (μ, σ, components)
   - transform() on test/production → apply learned parameters (no re-fitting)
   - Mathematical guarantee: Test data never influences training parameters

3. **✅ ColumnTransformer for Heterogeneous Data**
   - Different preprocessing for numeric vs categorical vs text columns
   - Parallel execution, concatenate outputs
   - Essential for real-world tabular data (80% of ML applications)

4. **✅ FeatureUnion for Diverse Representations**
   - Combine multiple feature extraction strategies (PCA + SelectKBest + Custom)
   - Capture different aspects of data (variance + correlation + domain knowledge)
   - Typical improvement: 5-15% accuracy over single strategy

5. **✅ Custom Transformers for Domain Expertise**
   - Inherit from BaseEstimator + TransformerMixin
   - fit() learns from training data, transform() applies learned parameters
   - Semiconductor examples: Spatial detrending, equipment drift correction

6. **✅ Production = Serialization + Versioning + Monitoring**
   - joblib for sklearn pipelines (fast, efficient)
   - Metadata JSON: Training date, hyperparameters, performance, dependencies
   - Monitoring: Input drift (3σ), prediction drift (10%), accuracy tracking

7. **✅ GridSearchCV with Pipelines**
   - Tune preprocessing + model hyperparameters jointly
   - Use caching to speed up expensive transforms (5-10x speedup)
   - Parameter naming: `step_name__param_name`

---

### **When to Use Each Component**

| **Component** | **Use Case** | **Example** |
|---------------|--------------|-------------|
| **Pipeline** | Sequential preprocessing + model | Scaler → PCA → Classifier |
| **ColumnTransformer** | Different transforms per column type | Numeric: scale, Categorical: encode |
| **FeatureUnion** | Combine multiple feature representations | PCA + SelectKBest |
| **Custom Transformer** | Domain-specific preprocessing | Wafer spatial detrending, drift correction |
| **FunctionTransformer** | Stateless transforms (no learned params) | np.log, np.sqrt |
| **make_pipeline** | Quick pipeline without naming steps | make_pipeline(Scaler(), PCA(), Model()) |

---

### **Common Pitfalls and Solutions**

| **Pitfall** | **Problem** | **Solution** |
|-------------|-------------|--------------|
| **Data leakage** | Fit scaler on train+test | Always fit only on train, transform on test |
| **Forgot to scale test** | Forgot scaler.transform(X_test) | Use Pipeline (automatic consistency) |
| **Version mismatch** | Trained on Python 3.8, load on 3.11 | Document dependencies, use Docker |
| **No monitoring** | Model degrades silently in production | Monitor input drift, prediction drift, accuracy |
| **No rollback** | Can't revert to previous version | Keep all model versions, use symlinks |
| **Slow GridSearchCV** | Expensive transforms repeated | Use caching with `memory` parameter |

---

### **Semiconductor-Specific Insights**

1. **Spatial detrending:** Essential for wafer map data (edge dies ≠ center dies)
2. **Multi-stage fusion:** Combine wafer test + final test data via FeatureUnion
3. **Real-time constraints:** <50ms latency for 1M devices/day
4. **High availability:** 99.9% uptime requirement ($1M/hour downtime cost)
5. **Audit trail:** Every prediction logged for regulatory compliance (ISO 26262)
6. **Cost of errors:** 5% yield loss = $10M-$50M annual impact

---

### **Production Checklist**

- [ ] Pipeline includes all preprocessing steps (no manual transforms)
- [ ] Serialized with joblib (or ONNX for cross-platform)
- [ ] Metadata JSON includes: version, date, hyperparameters, performance, dependencies
- [ ] Unit tests validate: Fit/transform correctness, output shapes, learned parameters
- [ ] Monitoring: Input drift, prediction drift, accuracy, latency
- [ ] Rollback strategy: Previous versions available, symlink for instant fallback
- [ ] Documentation: README with usage, API docs, troubleshooting
- [ ] CI/CD: Automated testing, performance validation, deployment

---

### **Next Steps**

1. **Practice:** Build end-to-end pipeline for your domain (text, images, tabular, time series)
2. **Experiment:** Try FeatureUnion with different combinations (PCA + SelectKBest + Custom)
3. **Deploy:** Serialize pipeline, deploy with Docker + FastAPI, monitor in production
4. **Optimize:** Use caching for expensive transforms, GridSearchCV for hyperparameters
5. **Learn more:** sklearn documentation, MLOps courses, production ML books

---

### **Resources**

**Documentation:**
- sklearn Pipeline: https://scikit-learn.org/stable/modules/compose.html
- ColumnTransformer: https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html
- FeatureUnion: https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.FeatureUnion.html

**Books:**
- "Building Machine Learning Pipelines" by Hannes Hapke & Catherine Nelson
- "Designing Machine Learning Systems" by Chip Huyen
- "Machine Learning Design Patterns" by Lakshmanan et al.

**Libraries:**
- sklearn: Standard ML library with Pipeline support
- joblib: Efficient serialization for numpy arrays
- ONNX: Cross-platform model deployment
- MLflow: Experiment tracking, model registry, deployment

---

**Congratulations!** 🎉 You now have comprehensive knowledge of ML pipelines and automation, from basic concepts to production deployment. Apply these skills to build robust, maintainable, production-ready ML systems!